# Task 1:  Strategic Design of Chatbot Analytics Framework

In [ ]:
# Data simulation for Task 1
import random
import uuid
import json
from datetime import datetime, timedelta
from collections import Counter, defaultdict
import math

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

random.seed(42)
np.random.seed(42)

# ----------------------------
# Configuration / Parameters
# ----------------------------
NUM_SESSIONS = 3000                   # number of synthetic sessions to simulate
MAX_MESSAGES_PER_SESSION = 8          # max messages per session (user+bot turns approx)
BASE_TIME = datetime.utcnow() - timedelta(days=30)  # simulate last 30 days
HUMAN_AGENT_HOURLY_COST = 25.0        # € per hour
AVG_HUMAN_CALL_DURATION_MIN = 7       # average length in minutes for a typical redirected call
DEVELOPMENT_COST = 5000.0
HOSTING_COST_PER_YEAR = 800.0
# Business assumptions for conversion value
AVG_CONVERSION_VALUE = 50.0           # € per successful conversion (e.g., account opening leads to avg value)

# --- Funnel-aware intent distribution with realistic drop-offs
INTENTS = [
    "greet", "check_balance", "transactions", "transfer_funds",
    "open_account", "loan_inquiry", "card_block", "faq_interest_rates",
    "update_details", "goodbye", "small_talk"
]

# Funnel progression probabilities (realistic drop-offs)
FUNNEL_PROBS = {
    "session_started": 1.0,           # 100% - all sessions start
    "greet_shown": 0.85,              # 85% - some users leave immediately
    "intent_expressed": 0.70,         # 70% - express a real intent
    "info_provided": 0.55,            # 55% - provide enough information
    "action_completed": 0.45,         # 35% - complete the desired action
    "resolved": 0.43                  # 25% - fully resolved without fallback
}

# intents that indicate a 'conversion' type business action (e.g., transfer, open_account)
CONVERSION_INTENTS = {"transfer_funds", "open_account"}

CHANNELS = ["web", "mobile_app", "whatsapp", "facebook_messenger"]

# probability distribution for true intents (should sum to 1)
# Adjusted to reflect funnel behavior - more common intents first
INTENT_PROBS = np.array([0.15, 0.20, 0.16, 0.12, 0.05, 0.07, 0.06, 0.08, 0.05, 0.03, 0.03])
INTENT_PROBS = INTENT_PROBS / INTENT_PROBS.sum()
FALLBACK_THRESHOLD = 0.55

# ----------------------------
# Helper functions
# ----------------------------
def pick_true_intent():
    return np.random.choice(INTENTS, p=INTENT_PROBS)

def simulate_predicted_intent(true_intent):
    """
    Simulate NLU predictions with realistic noise.
    Higher confidence for frequent intents; occasional misclassification.
    """
    # base confidence: high for most intents, lower for ambiguous ones
    base_conf = 0.88 if true_intent not in ("small_talk", "greet") else 0.75
    # add some randomness
    confidence = np.clip(np.random.normal(base_conf, 0.12), 0.2, 0.99)
    # misclassification probability inversely related to confidence
    misprob = 1 - confidence

    if random.random() < misprob:
        # pick a different intent (biased by frequency)
        other_intents = [i for i in INTENTS if i != true_intent]
        probs = np.array([INTENT_PROBS[INTENTS.index(i)] for i in other_intents])
        probs = probs / probs.sum()  # normalize
        predicted = np.random.choice(other_intents, p=probs)
    else:
        predicted = true_intent
    return predicted, round(float(confidence), 3)

def rand_response_time(channel):
    # target under 2000 ms (2s) for responsiveness, but simulate distribution
    if channel == "web":
        return int(np.clip(np.random.normal(350, 120), 80, 3000))
    elif channel == "mobile_app":
        return int(np.clip(np.random.normal(420, 150), 80, 4000))
    elif channel == "whatsapp":
        return int(np.clip(np.random.normal(600, 220), 80, 5000))
    else:
        return int(np.clip(np.random.normal(450, 180), 80, 4000))

def generate_session_with_funnel(start_time):
    """
    Generate session with realistic funnel progression
    """
    session_id = str(uuid.uuid4())
    user_id = f"user_{random.randint(1000, 99999)}"
    channel = random.choices(CHANNELS, weights=[0.45, 0.25, 0.18, 0.12])[0]

    messages = []
    funnel_stage = "session_started"
    used_fallback = False
    resolved = False
    conversion = False

    # Funnel progression simulation
    current_prob = 1.0

    # Stage 1: Session started -> Greet shown
    if random.random() < FUNNEL_PROBS["greet_shown"]:
        funnel_stage = "greet_shown"
        true_intent = "greet"
        predicted_intent, confidence = simulate_predicted_intent(true_intent)
        response_time_ms = rand_response_time(channel)
        timestamp = (start_time + timedelta(seconds=random.randint(1, 10))).isoformat()

        messages.append({
            "session_id": session_id, "user_id": user_id, "timestamp": timestamp,
            "channel": channel, "true_intent": true_intent,
            "predicted_intent": predicted_intent, "intent_confidence": confidence,
            "action": "action_greet", "response_time_ms": response_time_ms,
            "funnel_stage": funnel_stage
        })

        # Stage 2: Greet shown -> Intent expressed
        if random.random() < FUNNEL_PROBS["intent_expressed"]:
            funnel_stage = "intent_expressed"
            true_intent = pick_true_intent()
            predicted_intent, confidence = simulate_predicted_intent(true_intent)
            response_time_ms = rand_response_time(channel)
            timestamp = (start_time + timedelta(seconds=random.randint(11, 30))).isoformat()

            messages.append({
                "session_id": session_id, "user_id": user_id, "timestamp": timestamp,
                "channel": channel, "true_intent": true_intent,
                "predicted_intent": predicted_intent, "intent_confidence": confidence,
                "action": f"action_{predicted_intent}", "response_time_ms": response_time_ms,
                "funnel_stage": funnel_stage
            })

            # Stage 3: Intent expressed -> Info provided (additional messages)
            if random.random() < FUNNEL_PROBS["info_provided"]:
                funnel_stage = "info_provided"
                num_additional_messages = random.randint(1, 3)

                for i in range(num_additional_messages):
                    additional_intent = true_intent if random.random() < 0.7 else pick_true_intent()
                    predicted_intent, confidence = simulate_predicted_intent(additional_intent)
                    response_time_ms = rand_response_time(channel)
                    timestamp = (start_time + timedelta(seconds=random.randint(31, 60) + i*10)).isoformat()

                    # Check for fallback
                    if predicted_intent != additional_intent or confidence < FALLBACK_THRESHOLD:
                        if random.random() < 0.3:  # 30% chance of fallback when confidence low
                            used_fallback = True
                            predicted_intent = "fallback"

                    messages.append({
                        "session_id": session_id, "user_id": user_id, "timestamp": timestamp,
                        "channel": channel, "true_intent": additional_intent,
                        "predicted_intent": predicted_intent, "intent_confidence": confidence,
                        "action": f"action_{predicted_intent}", "response_time_ms": response_time_ms,
                        "funnel_stage": funnel_stage
                    })

                # Stage 4: Info provided -> Action completed
                if random.random() < FUNNEL_PROBS["action_completed"]:
                    funnel_stage = "action_completed"

                    # Final interaction
                    final_intent = "goodbye" if random.random() < 0.8 else true_intent
                    predicted_intent, confidence = simulate_predicted_intent(final_intent)
                    response_time_ms = rand_response_time(channel)
                    timestamp = (start_time + timedelta(seconds=random.randint(61, 120))).isoformat()

                    messages.append({
                        "session_id": session_id, "user_id": user_id, "timestamp": timestamp,
                        "channel": channel, "true_intent": final_intent,
                        "predicted_intent": predicted_intent, "intent_confidence": confidence,
                        "action": f"action_{predicted_intent}", "response_time_ms": response_time_ms,
                        "funnel_stage": funnel_stage
                    })

                    # Stage 5: Action completed -> Resolved
                    if random.random() < FUNNEL_PROBS["resolved"] and not used_fallback:
                        funnel_stage = "resolved"
                        resolved = True

                        # Check for conversion
                        if true_intent in CONVERSION_INTENTS and random.random() < 0.6:
                            conversion = True

    # Post-chat rating (simulate more likely if resolved)
    if random.random() < 0.65:
        base_rating = 4.3 if resolved else (3.8 if funnel_stage == "action_completed" else 3.0)
        rating = int(np.clip(round(np.random.normal(base_rating, 0.8)), 1, 5))
    else:
        rating = None

    # Add funnel stage to all messages
    for msg in messages:
        msg.update({
            "session_resolved": resolved,
            "session_used_fallback": used_fallback,
            "session_conversion": conversion,
            "post_rating": rating,
            "funnel_stage": funnel_stage
        })

    return messages, resolved, used_fallback, conversion, rating, channel

# ----------------------------
# Data generation
# ----------------------------
rows = []
session_meta = {}
funnel_tracking = defaultdict(int)

start_base = BASE_TIME
for i in range(NUM_SESSIONS):
    st = start_base + timedelta(seconds=random.randint(0, 30*24*3600))
    msgs, resolved, used_fallback, conversion, rating, ch = generate_session_with_funnel(st)

    # Track funnel progression
    if msgs:
        funnel_stage = msgs[0]["funnel_stage"]
        funnel_tracking[funnel_stage] += 1

        # Collect session-level meta
        session_meta[msgs[0]["session_id"]] = {
            "user_id": msgs[0]["user_id"],
            "channel": ch,
            "session_time": msgs[0]["timestamp"],
            "resolved": resolved,
            "used_fallback": used_fallback,
            "conversion": conversion,
            "post_rating": rating,
            "funnel_stage": funnel_stage
        }

        rows.extend(msgs)

df = pd.DataFrame(rows)
task1_df = df.copy()
print(f"Generated {df['session_id'].nunique()} sessions and {len(df)} messages. Sample:")
print(f"Funnel distribution: {dict(funnel_tracking)}")
display(df.head())

# Save logs as JSON and CSV
df.to_csv("/content/banking_chatbot_logs.csv", index=False)
with open("/content/banking_chatbot_logs.json", "w", encoding="utf-8") as f:
    json.dump(json.loads(df.to_json(orient="records")), f, indent=2, ensure_ascii=False)
print("Saved logs to /content/banking_chatbot_logs.csv and .json")


/tmp/ipython-input-44961425.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  BASE_TIME = datetime.utcnow() - timedelta(days=30)  # simulate last 30 days


Generated 2574 sessions and 6854 messages. Sample:
Funnel distribution: {'intent_expressed': 797, 'greet_shown': 765, 'resolved': 162, 'action_completed': 280, 'info_provided': 570}


,session_id,user_id,timestamp,channel,true_intent,predicted_intent,intent_confidence,action,response_time_ms,funnel_stage,session_resolved,session_used_fallback,session_conversion,post_rating
0,adb1f7cb-6460-48ff-b16e-6be4be0b15a9,user_4278,2025-09-25T06:36:16.960792,whatsapp,greet,card_block,0.810,action_greet,569,intent_expressed,False,False,False,4.0
1,adb1f7cb-6460-48ff-b16e-6be4be0b15a9,user_4278,2025-09-25T06:36:27.960792,whatsapp,transfer_funds,transfer_funds,0.852,action_transfer_funds,548,intent_expressed,False,False,False,4.0
2,c9a49f29-97ed-4a57-b412-b02ec771f52d,user_29657,2025-09-24T10:03:33.960792,web,greet,greet,0.842,action_greet,293,intent_expressed,False,False,False,2.0
3,c9a49f29-97ed-4a57-b412-b02ec771f52d,user_29657,2025-09-24T10:03:43.960792,web,check_balance,check_balance,0.945,action_check_balance,281,intent_expressed,False,False,False,2.0
4,65167005-d1e5-4640-90f8-05e09852b207,user_92506,2025-09-27T14:54:43.960792,web,greet,transactions,0.436,action_greet,464,intent_expressed,False,False,False,2.0


Saved logs to /content/banking_chatbot_logs.csv and .json


In [ ]:
# ----------------------------
# KPI computations (Task requirements)
# ----------------------------
def compute_intent_metrics(df):
    # compute per-intent precision, recall, f1 using counts of true vs predicted
    intents = sorted(list(set(df["true_intent"].unique()) | set(df["predicted_intent"].unique())))
    metrics = []
    for intent in intents:
        tp = len(df[(df["true_intent"]==intent) & (df["predicted_intent"]==intent)])
        fp = len(df[(df["true_intent"]!=intent) & (df["predicted_intent"]==intent)])
        fn = len(df[(df["true_intent"]==intent) & (df["predicted_intent"]!=intent)])
        precision = tp / (tp+fp) if (tp+fp)>0 else 0.0
        recall = tp / (tp+fn) if (tp+fn)>0 else 0.0
        f1 = 2*precision*recall/(precision+recall) if (precision+recall)>0 else 0.0
        metrics.append({"intent": intent, "tp": tp, "fp": fp, "fn": fn,
                        "precision": round(precision,3),"recall": round(recall,3),"f1": round(f1,3)})
    return pd.DataFrame(metrics).sort_values("f1", ascending=False)

intent_metrics_df = compute_intent_metrics(df)
print("\nIntent-level metrics (precision/recall/f1)")
display(intent_metrics_df.head(12))

# Overall intent accuracy (simple)
overall_accuracy = round((df["true_intent"]==df["predicted_intent"]).mean(),3)
print(f"Overall intent match accuracy: {overall_accuracy}")

# ----------------------------
# Micro and Macro F1
# ----------------------------
tp_global = intent_metrics_df["tp"].sum()
fp_global = intent_metrics_df["fp"].sum()
fn_global = intent_metrics_df["fn"].sum()
precision_micro = tp_global / (tp_global + fp_global) if (tp_global + fp_global) > 0 else 0.0
recall_micro = tp_global / (tp_global + fn_global) if (tp_global + fn_global) > 0 else 0.0
f1_micro = 2 * precision_micro * recall_micro / (precision_micro + recall_micro) if (precision_micro + recall_micro) > 0 else 0.0
f1_macro = intent_metrics_df["f1"].mean()
print(f"Micro F1: {round(f1_micro,3)}, Macro F1: {round(f1_macro,3)}")

# ----------------------------
# Response time KPIs
# ----------------------------
response_kpis = {
    "avg_response_time_ms": int(df["response_time_ms"].mean()),
    "median_response_time_ms": int(df["response_time_ms"].median()),
    "p95_response_time_ms": int(df["response_time_ms"].quantile(0.95))
}
print("\nResponse time KPIs (ms):", response_kpis)

# ----------------------------
# Session-level KPIs with deterministic fallback
# ----------------------------
session_df = pd.DataFrame.from_dict(session_meta, orient="index").reset_index().rename(columns={"index":"session_id"})

# Use deterministic fallback threshold
session_df["used_fallback"] = session_df["used_fallback"] | df.groupby("session_id")["intent_confidence"].min().lt(FALLBACK_THRESHOLD).reindex(session_df["session_id"]).fillna(False)

fallback_rate = round(session_df["used_fallback"].mean(), 3)
resolution_rate = round(session_df["resolved"].mean(), 3)
conversion_rate = round(session_df["conversion"].mean(), 3)
print(f"\nSession-level fallback_rate: {fallback_rate}; resolution_rate: {resolution_rate}; conversion_rate: {conversion_rate}")

# ----------------------------
# 30-day retention computation (cohort-based)
# ----------------------------
session_df['session_time_dt'] = pd.to_datetime(session_df['session_time'])
user_first_last = session_df.groupby('user_id')['session_time_dt'].agg(['min','max','count']).reset_index()
user_first_last['within_30d'] = (user_first_last['max'] - user_first_last['min']) <= pd.Timedelta(days=30)
retention_30d = (user_first_last['within_30d'] & (user_first_last['count']>1)).mean()
retention_30d = round(retention_30d, 3)
print(f"\n30-day retention (fraction of users with repeat sessions within 30 days): {retention_30d}")

# Average user rating
ratings = [v for v in session_df["post_rating"].tolist() if v is not None]
avg_rating = round(pd.Series(ratings).mean(),2) if len(ratings)>0 else None
print("Average post-chat rating (where provided):", avg_rating)

# ----------------------------
# Low-F1 intents table
# ----------------------------
LOW_F1_THRESHOLD = 0.65
low_f1_intents = intent_metrics_df[intent_metrics_df["f1"] < LOW_F1_THRESHOLD].sort_values("f1")
print(f"\nIntents with F1 < {LOW_F1_THRESHOLD} (need improvement):")
display(low_f1_intents[["intent","precision","recall","f1"]])

# ----------------------------
# Funnel Analysis
# ----------------------------
funnel_counts = {"All sessions": len(session_df)}
# greet step
greet_sessions = set(df[df["true_intent"]=="greet"]["session_id"].unique())
funnel_counts["Greet"] = len(greet_sessions)
# task selection (any service intent except greet/small_talk/goodbye)
service_intents = set(INTENTS) - {"greet","small_talk","goodbye"}
task_sessions = set(df[df["true_intent"].isin(service_intents)]["session_id"].unique())
funnel_counts["Task selected"] = len(task_sessions)
# provided info (>=3 turns heuristic)
provided_info_sessions = set([s for s,g in df.groupby("session_id") if len(g)>=3])
funnel_counts["Provided info (>=3 turns)"] = len(provided_info_sessions)
# completion (resolved)
funnel_counts["Completed (resolved)"] = int(session_df["resolved"].sum())

funnel_df = pd.DataFrame({"step": list(funnel_counts.keys()), "sessions": list(funnel_counts.values())})
print("\nFunnel counts:")
display(funnel_df)

# ----------------------------
# Business KPIs (Conversion, Retention, ROI estimate)
# ----------------------------
# Conversion rate computed earlier as conversion_rate
# Retention: fraction of users with >1 session in 30-day window (approx)
user_session_counts = session_df["user_id"].value_counts()
retained_users_fraction = round((user_session_counts>1).mean(),3)
print(f"\nEstimated user retention (repeat sessions within simulated window): {retained_users_fraction}")

# ROI estimate: basic example using rerouted calls saved
# assume sessions resolved by bot represent calls averted
calls_rerouted = int(session_df["resolved"].sum())
annual_savings = calls_rerouted * (AVG_HUMAN_CALL_DURATION_MIN/60) * HUMAN_AGENT_HOURLY_COST
total_cost = DEVELOPMENT_COST + HOSTING_COST_PER_YEAR
roi = round((annual_savings - total_cost) / total_cost * 100, 2)
print(f"\nEstimated annual savings (approx): €{int(annual_savings)}; ROI estimate: {roi}% (based on simplistic assumptions)")

# ----------------------------
# Visualizations (Plotly)
# ----------------------------
# Top intents bar
top_intents = df["true_intent"].value_counts().reset_index()
top_intents.columns = ["intent", "count"]
fig1 = px.bar(top_intents, x="intent", y="count", title="Top True Intents (by message count)", text="count")
fig1.update_layout(xaxis_title="Intent", yaxis_title="Message Count", height=400)
fig1.show()

# Response time histogram
fig2 = px.histogram(df, x="response_time_ms", nbins=60, title="Response Time Distribution (ms)", marginal="box")
fig2.update_layout(height=400)
fig2.show()

# Channel distribution pie (sessions)
chan = session_df["channel"].value_counts().reset_index()
chan.columns = ["channel", "sessions"]
fig3 = px.pie(chan, names="channel", values="sessions", title="Channel Distribution (sessions)")
fig3.show()

# Funnel chart
fig4 = go.Figure(go.Funnel(y=funnel_df["step"], x=funnel_df["sessions"], textinfo="value+percent initial"))
fig4.update_layout(title="Funnel Analysis: Banking Chatbot", height=450)
fig4.show()

# Intent confusion matrix (simplified heatmap of counts predicted vs true)
confusion = pd.crosstab(df["true_intent"], df["predicted_intent"])
fig5 = px.imshow(confusion.values, x=confusion.columns, y=confusion.index,
                 labels=dict(x="Predicted Intent", y="True Intent", color="Count"),
                 title="Intent Confusion Matrix (counts)")
fig5.update_layout(height=600)
fig5.show()

# ----------------------------
# Recommendations checklist (based on computed KPIs)
# ----------------------------
recommendations = []
if response_kpis["p95_response_time_ms"] > 2000:
    recommendations.append("Reduce P95 response time: optimize backend or use caching for frequent queries.")
if fallback_rate > 0.10:
    recommendations.append("Fallback rate exceeds 10% target: improve NLU training data and fallback handling.")
if overall_accuracy < 0.8:
    recommendations.append("Intent accuracy < 80%: retrain NLU with more labeled examples and active learning.")
if conversion_rate < 0.03:
    recommendations.append("Low conversion: review booking/transfer flows and simplify confirmation steps.")

print("\nRecommended next actions based on KPIs:")
for r in recommendations:
    print("-", r)

# Save KPIs and funnel to CSV for report use
kpis = {
    "overall_intent_accuracy": float(overall_accuracy),
    **response_kpis,
    "fallback_rate": float(fallback_rate),
    "resolution_rate": float(resolution_rate),
    "conversion_rate": float(conversion_rate),
    "avg_rating": float(avg_rating) if avg_rating is not None else None,
    "retention_estimate": float(retained_users_fraction),
    "estimated_annual_savings": int(annual_savings),
    "roi_percent": float(roi)
}
pd.DataFrame([kpis]).to_csv("/content/banking_chatbot_kpis.csv", index=False)
funnel_df.to_csv("/content/banking_chatbot_funnel.csv", index=False)
print("\nSaved KPI summary and funnel CSVs to /content/")

# Final summary print for quick copy-paste into report
print("\n--- TASK 1 SUMMARY ---")
print(f"Sessions simulated: {session_df.shape[0]}")
print(f"Overall intent match accuracy: {overall_accuracy}")
print(f"Fallback rate (session-level): {fallback_rate}")
print(f"Resolution rate (session-level): {resolution_rate}")
print(f"Conversion rate (session-level): {conversion_rate}")
print(f"Avg response time (ms): {response_kpis['avg_response_time_ms']} (p95: {response_kpis['p95_response_time_ms']})")
print(f"Avg user rating: {avg_rating}")
print(f"Estimated ROI (%): {roi}")



Intent-level metrics (precision/recall/f1)


,intent,tp,fp,fn,precision,recall,f1
4,goodbye,385,50,63,0.885,0.859,0.872
5,greet,2393,81,792,0.967,0.751,0.846
3,faq_interest_rates,300,103,53,0.744,0.850,0.794
9,transactions,580,214,90,0.730,0.866,0.792
0,card_block,186,73,30,0.718,0.861,0.783
1,check_balance,677,269,107,0.716,0.864,0.783
6,loan_inquiry,207,85,34,0.709,0.859,0.777
7,open_account,180,74,31,0.709,0.853,0.774
11,update_details,173,74,29,0.700,0.856,0.771
10,transfer_funds,380,156,72,0.709,0.841,0.769


Overall intent match accuracy: 0.807
Micro F1: 0.807, Macro F1: 0.717

Response time KPIs (ms): {'avg_response_time_ms': 431, 'median_response_time_ms': 409, 'p95_response_time_ms': 781}

Session-level fallback_rate: 0.033; resolution_rate: 0.063; conversion_rate: 0.006

30-day retention (fraction of users with repeat sessions within 30 days): 0.017
Average post-chat rating (where provided): 3.16

Intents with F1 < 0.65 (need improvement):


,intent,precision,recall,f1
2,fallback,0.000,0.000,0.000
8,small_talk,0.556,0.761,0.642



Funnel counts:


,step,sessions
0,All sessions,2574
1,Greet,2574
2,Task selected,1524
3,Provided info (>=3 turns),1012
4,Completed (resolved),162



Estimated user retention (repeat sessions within simulated window): 0.017

Estimated annual savings (approx): €472; ROI estimate: -91.85% (based on simplistic assumptions)



Recommended next actions based on KPIs:
- Low conversion: review booking/transfer flows and simplify confirmation steps.

Saved KPI summary and funnel CSVs to /content/

--- TASK 1 SUMMARY ---
Sessions simulated: 2574
Overall intent match accuracy: 0.807
Fallback rate (session-level): 0.033
Resolution rate (session-level): 0.063
Conversion rate (session-level): 0.006
Avg response time (ms): 431 (p95: 781)
Avg user rating: 3.16
Estimated ROI (%): -91.85


# TASK 3: Practical Implementation and Evaluation of Chatbot Analytics

In [ ]:
# Install required packages (if not already installed)
!pip install pandas numpy matplotlib seaborn plotly nltk textblob scikit-learn transformers torch
!pip install wordcloud spacy textstat
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.2/239.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 110.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re
import json
from datetime import datetime, timedelta
import random

# NLP Libraries
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from textblob import TextBlob
import spacy
import textstat
# ML and Analytics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import silhouette_score
# Download NLTK data
nltk.download(['vader_lexicon', 'punkt', 'stopwords', 'averaged_perceptron_tagger'])

# %%
# %% [markdown]
# ## 1. ENHANCED DATA GENERATION WITH ALL SEGMENTATION FEATURES

# %%
class CompleteTravelChatbotDataGenerator:
    def __init__(self):
        self.destinations = ['Paris', 'London', 'Tokyo', 'New York', 'Bali', 'Dubai', 'Rome', 'Sydney', 'Bangkok', 'Barcelona']
        self.hotels = ['Hilton', 'Marriott', 'Hyatt', 'Sheraton', 'InterContinental', 'Ritz Carlton', 'Four Seasons']
        self.airlines = ['Emirates', 'Singapore Airlines', 'Qatar Airways', 'British Airways', 'Lufthansa', 'Delta']
        self.intents = ['flight_booking', 'hotel_booking', 'destination_info', 'visa_requirements', 'weather_info', 'budget_planning', 'cancellation']

        # Enhanced segmentation attributes
        self.channels = ['WhatsApp', 'Web', 'Mobile_App', 'Facebook_Messenger']
        self.locations = ['USA', 'UK', 'India', 'Germany', 'Australia', 'Japan', 'Brazil', 'France']
        self.languages = ['English', 'Spanish', 'French', 'German', 'Japanese', 'Hindi']

    def generate_conversation(self, conversation_id):
        """Generate synthetic conversation data with enhanced segmentation"""
        user_profiles = [
            {'age': 25, 'user_type': 'backpacker', 'budget': 'low'},
            {'age': 35, 'user_type': 'business', 'budget': 'high'},
            {'age': 45, 'user_type': 'family', 'budget': 'medium'},
            {'age': 28, 'user_type': 'couple', 'budget': 'medium'}
        ]

        profile = random.choice(user_profiles)
        start_time = datetime.now() - timedelta(days=random.randint(1, 30))

        # Enhanced: Add channel, location, login status, language
        channel = random.choice(self.channels)
        location = random.choice(self.locations)
        login_status = random.choice([True, False])  # True = logged in, False = guest
        language = random.choice(self.languages)

        conversation = {
            'conversation_id': conversation_id,
            'user_id': f"user_{random.randint(1000, 9999)}",
            'user_age': profile['age'],
            'user_type': profile['user_type'],
            'budget_category': profile['budget'],
            'channel': channel,
            'location': location,
            'login_status': login_status,
            'language': language,
            'start_time': start_time,
            'messages': [],
            'successful_booking': random.choice([True, False]),
            'user_rating': random.randint(1, 5) if random.random() > 0.3 else None
        }

        # Generate message sequence
        num_messages = random.randint(4, 12)
        current_time = start_time

        # Initial greeting
        greetings = ["Hi, I need help planning a trip", "Hello, I want to book a vacation", "Hey, looking for travel suggestions"]
        conversation['messages'].append({
            'timestamp': current_time,
            'sender': 'user',
            'message': random.choice(greetings),
            'intent': 'greeting'
        })
        current_time += timedelta(minutes=1)

        # Conversation flow
        for i in range(num_messages - 2):
            if i % 2 == 0:  # User messages
                user_msg = self.generate_user_message(conversation)
                conversation['messages'].append({
                    'timestamp': current_time,
                    'sender': 'user',
                    'message': user_msg['text'],
                    'intent': user_msg['intent']
                })
            else:  # Bot responses
                bot_msg = self.generate_bot_response(conversation['messages'][-1])
                conversation['messages'].append({
                    'timestamp': current_time,
                    'sender': 'bot',
                    'message': bot_msg['text'],
                    'intent': bot_msg['intent']
                })
            current_time += timedelta(minutes=random.randint(1, 5))

        # Final message
        endings = ["Thanks, that helps!", "I'll think about it", "Great, thanks for the info!"]
        conversation['messages'].append({
            'timestamp': current_time,
            'sender': 'user',
            'message': random.choice(endings),
            'intent': 'closing'
        })

        return conversation

    def generate_user_message(self, conversation):
        """Generate user message based on conversation context"""
        intents_weights = {
            'flight_booking': 0.25,
            'hotel_booking': 0.2,
            'destination_info': 0.15,
            'visa_requirements': 0.1,
            'weather_info': 0.1,
            'budget_planning': 0.1,
            'cancellation': 0.05
        }

        intent = random.choices(list(intents_weights.keys()), weights=list(intents_weights.values()))[0]

        messages = {
            'flight_booking': [
                f"I want to book a flight to {random.choice(self.destinations)}",
                f"Find me cheap flights to {random.choice(self.destinations)}",
                f"Flight options for {random.choice(self.destinations)} next month"
            ],
            'hotel_booking': [
                f"Need hotel in {random.choice(self.destinations)} for {random.randint(2, 7)} nights",
                f"Find {random.choice(['budget', 'luxury'])} hotels in {random.choice(self.destinations)}",
                f"Hotel booking for {random.randint(1, 4)} people"
            ],
            'destination_info': [
                f"Tell me about {random.choice(self.destinations)}",
                f"What to do in {random.choice(self.destinations)}?",
                f"Is {random.choice(self.destinations)} good for families?"
            ]
        }

        return {
            'text': random.choice(messages.get(intent, ["I need travel help"])),
            'intent': intent
        }

    def generate_bot_response(self, last_message):
        """Generate bot response based on user message"""
        intent = last_message['intent']

        responses = {
            'flight_booking': [
                f"I found flights with {random.choice(self.airlines)} starting from ${random.randint(300, 1200)}",
                f"Here are the best flight options for your dates",
                f"Multiple airlines serve that route. When do you want to travel?"
            ],
            'hotel_booking': [
                f"I recommend {random.choice(self.hotels)} with {random.randint(3, 5)} star rating",
                f"Found several hotels in your budget range",
                f"Great choice! Hotels available from ${random.randint(80, 400)}/night"
            ],
            'destination_info': [
                f"It's a wonderful destination with many attractions",
                f"Great for {random.choice(['sightseeing', 'adventure', 'relaxation'])}",
                f"Popular activities include visiting famous landmarks and local cuisine"
            ]
        }

        return {
            'text': random.choice(responses.get(intent, ["How can I help you with your travel plans?"])),
            'intent': f"respond_to_{intent}"
        }

# %%
# Generate complete dataset
print("🔄 Generating complete travel chatbot data with ALL segmentation features...")
complete_generator = CompleteTravelChatbotDataGenerator()
complete_conversations = [complete_generator.generate_conversation(i) for i in range(500)]

# Convert to DataFrame
complete_rows = []
for conv in complete_conversations:
    for msg in conv['messages']:
        complete_rows.append({
            'conversation_id': conv['conversation_id'],
            'user_id': conv['user_id'],
            'user_age': conv['user_age'],
            'user_type': conv['user_type'],
            'budget_category': conv['budget_category'],
            'channel': conv['channel'],
            'location': conv['location'],
            'login_status': conv['login_status'],
            'language': conv['language'],
            'timestamp': msg['timestamp'],
            'sender': msg['sender'],
            'message': msg['message'],
            'intent': msg['intent'],
            'successful_booking': conv['successful_booking'],
            'user_rating': conv['user_rating']
        })

df = pd.DataFrame(complete_rows)
print(f"✅ Generated {len(df)} messages from {len(complete_conversations)} conversations")
print(f"📊 Complete dataset shape: {df.shape}")

# Display all segmentation columns
print("\n🎯 ALL SEGMENTATION COLUMNS AVAILABLE:")
print(f"• User Types: {df['user_type'].unique()}")
print(f"• Budget Categories: {df['budget_category'].unique()}")
print(f"• Channels: {df['channel'].unique()}")
print(f"• Locations: {df['location'].unique()}")
print(f"• Login Status: {df['login_status'].unique()}")
print(f"• Languages: {df['language'].unique()}")

# %%
# %% [markdown]
# ## 2. BASIC ANALYTICS & PERFORMANCE METRICS (ORIGINAL FEATURES)

# %%
class ChatbotAnalytics:
    def __init__(self, dataframe):
        self.df = dataframe
        self.sia = SentimentIntensityAnalyzer()
        self.nlp = spacy.load("en_core_web_sm")

    def calculate_basic_metrics(self):
        """Calculate fundamental chatbot performance metrics"""
        metrics = {}

        # Conversation metrics
        total_conversations = self.df['conversation_id'].nunique()
        metrics['total_conversations'] = total_conversations

        # Message metrics
        total_messages = len(self.df)
        user_messages = len(self.df[self.df['sender'] == 'user'])
        bot_messages = len(self.df[self.df['sender'] == 'bot'])

        metrics['total_messages'] = total_messages
        metrics['user_messages'] = user_messages
        metrics['bot_messages'] = bot_messages
        metrics['messages_per_conversation'] = total_messages / total_conversations

        # Success metrics
        successful_conversations = self.df[self.df['successful_booking'] == True]['conversation_id'].nunique()
        metrics['success_rate'] = (successful_conversations / total_conversations) * 100

        # Response time analysis (simulated)
        conversation_durations = []
        for conv_id in self.df['conversation_id'].unique():
            conv_data = self.df[self.df['conversation_id'] == conv_id]
            if len(conv_data) > 1:
                start = conv_data['timestamp'].min()
                end = conv_data['timestamp'].max()
                duration = (end - start).total_seconds() / 60  # in minutes
                conversation_durations.append(duration)

        metrics['avg_conversation_duration_min'] = np.mean(conversation_durations)
        metrics['median_conversation_duration_min'] = np.median(conversation_durations)

        # User rating metrics
        rated_conversations = self.df[self.df['user_rating'].notna()]
        if len(rated_conversations) > 0:
            metrics['avg_user_rating'] = rated_conversations['user_rating'].mean()
            metrics['rating_response_rate'] = (len(rated_conversations) / total_conversations) * 100
        else:
            metrics['avg_user_rating'] = 0
            metrics['rating_response_rate'] = 0

        return metrics

    def intent_analysis(self):
        """Analyze conversation intents and patterns"""
        intent_df = self.df[self.df['sender'] == 'user'].groupby('intent').agg({
            'conversation_id': 'nunique',
            'message': 'count',
            'successful_booking': 'mean'
        }).rename(columns={
            'conversation_id': 'conversation_count',
            'message': 'message_count',
            'successful_booking': 'success_rate'
        }).sort_values('conversation_count', ascending=False)

        intent_df['success_rate'] = intent_df['success_rate'] * 100
        return intent_df

    def user_behavior_analysis(self):
        """Analyze user behavior patterns"""
        user_analysis = self.df[self.df['sender'] == 'user'].groupby('user_type').agg({
            'conversation_id': 'nunique',
            'message': 'count',
            'successful_booking': 'mean',
            'user_rating': 'mean'
        }).rename(columns={
            'conversation_id': 'total_conversations',
            'message': 'total_messages',
            'successful_booking': 'booking_success_rate',
            'user_rating': 'avg_rating'
        })

        user_analysis['booking_success_rate'] = user_analysis['booking_success_rate'] * 100
        return user_analysis

# %%
# Initialize analytics and calculate metrics
print("📈 Calculating basic chatbot performance metrics...")
analytics = ChatbotAnalytics(df)
basic_metrics = analytics.calculate_basic_metrics()
intent_analysis = analytics.intent_analysis()
user_behavior = analytics.user_behavior_analysis()

# Display metrics
print("\n" + "="*50)
print("🤖 BASIC CHATBOT PERFORMANCE METRICS")
print("="*50)
for metric, value in basic_metrics.items():
    if 'rate' in metric or 'rating' in metric:
        print(f"{metric.replace('_', ' ').title()}: {value:.2f}%")
    elif 'duration' in metric:
        print(f"{metric.replace('_', ' ').title()}: {value:.2f} min")
    else:
        print(f"{metric.replace('_', ' ').title()}: {value:}")

# %%
# %% [markdown]
# ## 3. ADVANCED NLP & SENTIMENT ANALYSIS (ORIGINAL FEATURES)

# %%
class AdvancedNLPAnalytics:
    def __init__(self):
        self.sia = SentimentIntensityAnalyzer()
        self.nlp = spacy.load("en_core_web_sm")

    def analyze_sentiment(self, text):
        """Perform comprehensive sentiment analysis"""
        # VADER sentiment
        vader_scores = self.sia.polarity_scores(text)

        # TextBlob sentiment
        blob = TextBlob(text)
        textblob_polarity = blob.sentiment.polarity
        textblob_subjectivity = blob.sentiment.subjectivity

        # Compound sentiment score
        compound_score = (vader_scores['compound'] + textblob_polarity) / 2

        return {
            'vader_compound': vader_scores['compound'],
            'vader_positive': vader_scores['pos'],
            'vader_negative': vader_scores['neg'],
            'vader_neutral': vader_scores['neu'],
            'textblob_polarity': textblob_polarity,
            'textblob_subjectivity': textblob_subjectivity,
            'compound_sentiment': compound_score,
            'sentiment_label': 'positive' if compound_score > 0.05 else 'negative' if compound_score < -0.05 else 'neutral'
        }

    def calculate_message_complexity(self, text):
        """Calculate various text complexity metrics"""
        try:
            return {
                'readability_score': textstat.flesch_reading_ease(text),
                'grade_level': textstat.flesch_kincaid_grade(text),
                'word_count': len(text.split()),
                'sentence_count': len(sent_tokenize(text)),
                'avg_word_length': np.mean([len(word) for word in text.split()]) if text.split() else 0
            }
        except:
            return {
                'readability_score': 0,
                'grade_level': 0,
                'word_count': 0,
                'sentence_count': 0,
                'avg_word_length': 0
            }

    def extract_entities(self, text):
        """Extract named entities using spaCy"""
        doc = self.nlp(text)
        entities = {
            'locations': [],
            'organizations': [],
            'dates': [],
            'money': [],
            'other': []
        }

        for ent in doc.ents:
            if ent.label_ in ['GPE', 'LOC']:
                entities['locations'].append(ent.text)
            elif ent.label_ in ['ORG']:
                entities['organizations'].append(ent.text)
            elif ent.label_ in ['DATE', 'TIME']:
                entities['dates'].append(ent.text)
            elif ent.label_ in ['MONEY']:
                entities['money'].append(ent.text)
            else:
                entities['other'].append(ent.text)

        return entities

# %%
# Apply advanced NLP analysis
print("🔍 Performing advanced NLP analysis...")
nlp_analyzer = AdvancedNLPAnalytics()

# Sample analysis on user messages
user_messages = df[df['sender'] == 'user'].sample(min(1000, len(df[df['sender'] == 'user'])), random_state=42)

# Sentiment analysis
sentiment_results = []
complexity_results = []
entity_results = []

for idx, message in user_messages.iterrows():
    # Sentiment analysis
    sentiment = nlp_analyzer.analyze_sentiment(message['message'])
    sentiment['conversation_id'] = message['conversation_id']
    sentiment['message'] = message['message']
    sentiment_results.append(sentiment)

    # Complexity analysis
    complexity = nlp_analyzer.calculate_message_complexity(message['message'])
    complexity['conversation_id'] = message['conversation_id']
    complexity_results.append(complexity)

    # Entity extraction
    entities = nlp_analyzer.extract_entities(message['message'])
    entity_results.append({
        'conversation_id': message['conversation_id'],
        'entities': entities
    })

# Create DataFrames
sentiment_df = pd.DataFrame(sentiment_results)
complexity_df = pd.DataFrame(complexity_results)

print("✅ Advanced NLP analysis completed!")

# Display sentiment analysis results
print("\n" + "="*50)
print("😊 SENTIMENT ANALYSIS RESULTS")
print("="*50)
print(f"Overall Sentiment Distribution:")
print(sentiment_df['sentiment_label'].value_counts())
print(f"\nAverage Compound Sentiment Score: {sentiment_df['compound_sentiment'].mean():.3f}")

# %%
# %% [markdown]
# ## 4. ENHANCED SEGMENTATION ANALYTICS (NEW FEATURES)

# %%
class EnhancedSegmentationAnalytics:
    def __init__(self, dataframe):
        self.df = dataframe

    def channel_usage_analysis(self):
        """Analyze performance across different channels"""
        channel_analysis = self.df.groupby('channel').agg({
            'conversation_id': 'nunique',
            'message': 'count',
            'successful_booking': 'mean',
            'user_rating': 'mean',
            'user_age': 'mean'
        }).rename(columns={
            'conversation_id': 'total_conversations',
            'message': 'total_messages',
            'successful_booking': 'booking_success_rate',
            'user_rating': 'avg_rating',
            'user_age': 'avg_age'
        }).round(3)

        channel_analysis['booking_success_rate'] = channel_analysis['booking_success_rate'] * 100
        channel_analysis['messages_per_conversation'] = channel_analysis['total_messages'] / channel_analysis['total_conversations']

        return channel_analysis.sort_values('total_conversations', ascending=False)

    def location_based_analysis(self):
        """Analyze user behavior by geographic location"""
        location_analysis = self.df.groupby('location').agg({
            'conversation_id': 'nunique',
            'successful_booking': 'mean',
            'user_rating': 'mean',
            'user_age': 'mean',
            'budget_category': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Unknown'
        }).rename(columns={
            'conversation_id': 'total_conversations',
            'successful_booking': 'booking_success_rate',
            'user_rating': 'avg_rating',
            'user_age': 'avg_age',
            'budget_category': 'most_common_budget'
        }).round(3)

        location_analysis['booking_success_rate'] = location_analysis['booking_success_rate'] * 100

        return location_analysis.sort_values('total_conversations', ascending=False)

    def login_status_analysis(self):
        """Compare performance between logged-in vs guest users"""
        login_analysis = self.df.groupby('login_status').agg({
            'conversation_id': 'nunique',
            'successful_booking': 'mean',
            'user_rating': 'mean',
            'user_age': 'mean'
        }).rename(columns={
            'conversation_id': 'total_conversations',
            'successful_booking': 'booking_success_rate',
            'user_rating': 'avg_rating',
            'user_age': 'avg_age'
        }).round(3)

        login_analysis['booking_success_rate'] = login_analysis['booking_success_rate'] * 100
        login_analysis.index = login_analysis.index.map({True: 'Logged-in', False: 'Guest'})

        return login_analysis

    def language_preference_analysis(self):
        """Analyze user behavior by language preference"""
        language_analysis = self.df.groupby('language').agg({
            'conversation_id': 'nunique',
            'successful_booking': 'mean',
            'user_rating': 'mean',
            'location': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Unknown'
        }).rename(columns={
            'conversation_id': 'total_conversations',
            'successful_booking': 'booking_success_rate',
            'user_rating': 'avg_rating',
            'location': 'most_common_location'
        }).round(3)

        language_analysis['booking_success_rate'] = language_analysis['booking_success_rate'] * 100

        return language_analysis.sort_values('total_conversations', ascending=False)

# %%
# Perform enhanced segmentation analysis
print("🔍 Performing enhanced segmentation analysis...")
enhanced_analytics = EnhancedSegmentationAnalytics(df)

channel_analysis = enhanced_analytics.channel_usage_analysis()
location_analysis = enhanced_analytics.location_based_analysis()
login_analysis = enhanced_analytics.login_status_analysis()
language_analysis = enhanced_analytics.language_preference_analysis()

print("\n" + "="*60)
print("📱 CHANNEL USAGE ANALYSIS")
print("="*60)
print(channel_analysis)

print("\n" + "="*60)
print("🌍 LOCATION-BASED ANALYSIS")
print("="*60)
print(location_analysis.head(8))

print("\n" + "="*60)
print("🔐 LOGIN STATUS ANALYSIS")
print("="*60)
print(login_analysis)

print("\n" + "="*60)
print("🗣️ LANGUAGE PREFERENCE ANALYSIS")
print("="*60)
print(language_analysis)

# %%
# %% [markdown]
# ## 5. COMPREHENSIVE VISUALIZATION DASHBOARD (ALL FEATURES)

# %%
class CompleteChatbotVisualizer:
    def __init__(self):
        self.color_palette = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD', '#87CEEB']

    def create_comprehensive_dashboard(self, metrics, intent_analysis, user_behavior,
                                    channel_analysis, location_analysis, login_analysis,
                                    sentiment_df):
        """Create comprehensive performance dashboard with ALL features"""
        fig = make_subplots(
            rows=3, cols=3,
            subplot_titles=(
                'Conversation Success Rate', 'User Rating Distribution', 'Intent Frequency',
                'Messages by User Type', 'Channel Performance', 'Location Distribution',
                'Login Status Comparison', 'Sentiment Analysis', 'Language Distribution'
            ),
            specs=[
                [{"type": "pie"}, {"type": "bar"}, {"type": "bar"}],
                [{"type": "bar"}, {"type": "bar"}, {"type": "pie"}],
                [{"type": "bar"}, {"type": "pie"}, {"type": "pie"}]
            ]
        )

        # 1. Success rate pie chart
        success_rate = metrics['success_rate']
        fig.add_trace(
            go.Pie(labels=['Successful', 'Unsuccessful'],
                  values=[success_rate, 100-success_rate],
                  marker_colors=['#4ECDC4', '#FF6B6B']),
            row=1, col=1
        )

        # 2. User rating distribution
        ratings = df[df['user_rating'].notna()]['user_rating'].value_counts().sort_index()
        fig.add_trace(
            go.Bar(x=ratings.index, y=ratings.values, marker_color='#45B7D1'),
            row=1, col=2
        )

        # 3. Top intents
        top_intents = intent_analysis.head(5)
        fig.add_trace(
            go.Bar(x=top_intents.index, y=top_intents['conversation_count'], marker_color='#96CEB4'),
            row=1, col=3
        )

        # 4. Messages by user type
        fig.add_trace(
            go.Bar(x=user_behavior.index, y=user_behavior['total_messages'], marker_color='#FFEAA7'),
            row=2, col=1
        )

        # 5. Channel performance (success rate)
        fig.add_trace(
            go.Bar(x=channel_analysis.index, y=channel_analysis['booking_success_rate'],
                  marker_color='#DDA0DD'),
            row=2, col=2
        )

        # 6. Location distribution
        location_counts = location_analysis.head(6)['total_conversations']
        fig.add_trace(
            go.Pie(labels=location_counts.index, values=location_counts.values,
                  marker_colors=px.colors.qualitative.Set3),
            row=2, col=3
        )

        # 7. Login status comparison
        fig.add_trace(
            go.Bar(x=login_analysis.index, y=login_analysis['booking_success_rate'],
                  marker_color=['#4ECDC4', '#FF6B6B']),
            row=3, col=1
        )

        # 8. Sentiment distribution
        sentiment_counts = sentiment_df['sentiment_label'].value_counts()
        fig.add_trace(
            go.Pie(labels=sentiment_counts.index, values=sentiment_counts.values,
                  marker_colors=['#4ECDC4', '#FFEAA7', '#FF6B6B']),
            row=3, col=2
        )

        # 9. Language distribution
        language_counts = language_analysis.head(5)['total_conversations']
        fig.add_trace(
            go.Pie(labels=language_counts.index, values=language_counts.values,
                  marker_colors=px.colors.qualitative.Pastel),
            row=3, col=3
        )

        fig.update_layout(height=900, showlegend=False,
                         title_text="COMPREHENSIVE CHATBOT ANALYTICS DASHBOARD")
        fig.show()

    def create_segmentation_detailed_view(self, channel_analysis, location_analysis, login_analysis, language_analysis):
        """Create detailed segmentation analysis view"""
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Channel Usage & Performance', 'Geographic Performance',
                'Logged-in vs Guest Analysis', 'Language Preference & Success'
            )
        )

        # Channel analysis
        fig.add_trace(
            go.Bar(x=channel_analysis.index, y=channel_analysis['total_conversations'],
                  name='Conversations', marker_color='#4ECDC4'),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=channel_analysis.index, y=channel_analysis['booking_success_rate'],
                      name='Success Rate %', mode='lines+markers', line=dict(color='#FF6B6B')),
            row=1, col=1
        )

        # Geographic performance
        top_locations = location_analysis.head(6)
        fig.add_trace(
            go.Bar(x=top_locations.index, y=top_locations['booking_success_rate'],
                  marker_color=px.colors.sequential.Viridis),
            row=1, col=2
        )

        # Login status detailed
        metrics = ['total_conversations', 'booking_success_rate', 'avg_rating']
        for i, metric in enumerate(metrics):
            fig.add_trace(
                go.Bar(x=[metric], y=[login_analysis.loc['Logged-in', metric]],
                      name='Logged-in', marker_color='#4ECDC4', showlegend=(i==0)),
                row=2, col=1
            )
            fig.add_trace(
                go.Bar(x=[metric], y=[login_analysis.loc['Guest', metric]],
                      name='Guest', marker_color='#FF6B6B', showlegend=(i==0)),
                row=2, col=1
            )

        # Language analysis
        top_languages = language_analysis.head(5)
        fig.add_trace(
            go.Bar(x=top_languages.index, y=top_languages['booking_success_rate'],
                  marker_color=px.colors.qualitative.Set3),
            row=2, col=2
        )

        fig.update_layout(height=700, title_text="Detailed Segmentation Analysis")
        fig.show()

# %%
# Create comprehensive visualizations
print("📊 Generating comprehensive analytics dashboard...")
complete_viz = CompleteChatbotVisualizer()

# Main dashboard
complete_viz.create_comprehensive_dashboard(
    basic_metrics, intent_analysis, user_behavior,
    channel_analysis, location_analysis, login_analysis,
    sentiment_df
)

# Detailed segmentation view
complete_viz.create_segmentation_detailed_view(
    channel_analysis, location_analysis, login_analysis, language_analysis
)

# %%
# %% [markdown]
# ## 6. TOPIC MODELING & CONVERSATION CLUSTERING (ORIGINAL FEATURES)

# %%
class TopicModelingAnalyzer:
    def __init__(self):
        self.vectorizer = TfidfVectorizer(max_features=100, stop_words='english')
        self.lda = LatentDirichletAllocation(n_components=5, random_state=42)

    def analyze_conversation_topics(self, df):
        """Perform topic modeling on user messages"""
        user_messages = df[df['sender'] == 'user']
        messages = user_messages['message'].tolist()

        # Create TF-IDF matrix
        tfidf_matrix = self.vectorizer.fit_transform(messages)

        # Apply LDA
        lda_matrix = self.lda.fit_transform(tfidf_matrix)

        # Get feature names
        feature_names = self.vectorizer.get_feature_names_out()

        # Display topics
        topics = {}
        for topic_idx, topic in enumerate(self.lda.components_):
            top_features = [feature_names[i] for i in topic.argsort()[:-10 - 1:-1]]
            topics[f"Topic_{topic_idx}"] = top_features

        return topics, lda_matrix

# %%
# Perform topic modeling
print("🔮 Performing topic modeling...")
topic_analyzer = TopicModelingAnalyzer()
topics, lda_matrix = topic_analyzer.analyze_conversation_topics(df)

print("\n" + "="*50)
print("📚 DISCOVERED CONVERSATION TOPICS")
print("="*50)
for topic, words in topics.items():
    print(f"{topic}: {', '.join(words[:5])}")

# %%
# %% [markdown]
# ## 7. COMPREHENSIVE OPTIMIZATION RECOMMENDATIONS (ALL FEATURES)

# %%
class CompleteOptimizationRecommender:
    def __init__(self, df, metrics, intent_analysis, sentiment_df,
                 channel_analysis, location_analysis, login_analysis, language_analysis):
        self.df = df
        self.metrics = metrics
        self.intent_analysis = intent_analysis
        self.sentiment_df = sentiment_df
        self.channel_analysis = channel_analysis
        self.location_analysis = location_analysis
        self.login_analysis = login_analysis
        self.language_analysis = language_analysis

    def generate_comprehensive_recommendations(self):
        """Generate comprehensive data-driven optimization recommendations"""
        recommendations = []

        # 1. Basic performance recommendations
        if self.metrics['success_rate'] < 60:
            recommendations.append({
                'category': 'Success Rate',
                'issue': f"Low booking success rate ({self.metrics['success_rate']:.1f}%)",
                'recommendation': "Implement better intent recognition and provide more personalized options",
                'priority': 'High',
                'impact': 'Direct revenue impact'
            })

        # 2. Conversation efficiency
        if self.metrics['messages_per_conversation'] > 8:
            recommendations.append({
                'category': 'Efficiency',
                'issue': f"Long conversations ({self.metrics['messages_per_conversation']:.1f} messages avg)",
                'recommendation': "Streamline booking process with fewer steps and pre-filled options",
                'priority': 'Medium',
                'impact': 'User experience improvement'
            })

        # 3. Intent-based optimization
        failed_intents = self.intent_analysis[self.intent_analysis['success_rate'] < 50]
        if not failed_intents.empty:
            for intent, row in failed_intents.head(3).iterrows():
                recommendations.append({
                    'category': 'Intent Handling',
                    'issue': f"Poor success rate for '{intent}' intent ({row['success_rate']:.1f}%)",
                    'recommendation': f"Improve training data and response accuracy for {intent} requests",
                    'priority': 'High',
                    'impact': 'User satisfaction'
                })

        # 4. Sentiment-based optimization
        negative_sentiment_rate = (len(self.sentiment_df[self.sentiment_df['sentiment_label'] == 'negative']) /
                                 len(self.sentiment_df)) * 100
        if negative_sentiment_rate > 20:
            recommendations.append({
                'category': 'User Sentiment',
                'issue': f"High negative sentiment rate ({negative_sentiment_rate:.1f}%)",
                'recommendation': "Implement real-time sentiment analysis to route frustrated users to human agents",
                'priority': 'Medium',
                'impact': 'Customer retention'
            })

        # 5. Channel-specific optimization
        underperforming_channels = self.channel_analysis[self.channel_analysis['booking_success_rate'] < 60]
        for channel, row in underperforming_channels.iterrows():
            recommendations.append({
                'category': 'Channel Optimization',
                'issue': f"Low success rate on {channel} ({row['booking_success_rate']:.1f}%)",
                'recommendation': f"Optimize {channel} interface and implement channel-specific response templates",
                'priority': 'Medium',
                'impact': 'Channel performance improvement'
            })

        # 6. Location-based optimization
        underperforming_locations = self.location_analysis[self.location_analysis['booking_success_rate'] < 50]
        for location, row in underperforming_locations.head(3).iterrows():
            recommendations.append({
                'category': 'Geographic Optimization',
                'issue': f"Poor performance from {location} ({row['booking_success_rate']:.1f}%)",
                'recommendation': f"Localize content and offers for {location} users, add local payment methods",
                'priority': 'Medium',
                'impact': 'Market-specific growth'
            })

        # 7. Login status optimization
        guest_success = self.login_analysis.loc['Guest', 'booking_success_rate']
        logged_in_success = self.login_analysis.loc['Logged-in', 'booking_success_rate']
        if guest_success < logged_in_success * 0.8:
            recommendations.append({
                'category': 'User Conversion',
                'issue': f"Guest users underperform ({guest_success:.1f}% vs {logged_in_success:.1f}% for logged-in)",
                'recommendation': "Implement guest-to-logged-in conversion incentives and simplified registration",
                'priority': 'High',
                'impact': 'User retention and conversion'
            })

        # 8. Language optimization
        underperforming_languages = self.language_analysis[self.language_analysis['booking_success_rate'] < 50]
        for language, row in underperforming_languages.iterrows():
            recommendations.append({
                'category': 'Localization',
                'issue': f"Poor performance for {language} speakers ({row['booking_success_rate']:.1f}%)",
                'recommendation': f"Improve {language} language support and cultural localization",
                'priority': 'Medium',
                'impact': 'International user satisfaction'
            })

        return recommendations

    def generate_personalization_strategies(self):
        """Generate comprehensive user-centric personalization strategies"""
        strategies = []

        # User type personalization
        for user_type in self.df['user_type'].unique():
            user_data = self.df[self.df['user_type'] == user_type]
            success_rate = user_data['successful_booking'].mean() * 100

            if success_rate < 60:
                strategies.append({
                    'segment': f"{user_type.title()} Users",
                    'strategy': f"Develop {user_type}-specific conversation flows and recommendations",
                    'key_actions': [
                        f"Create {user_type}-tailored destination suggestions",
                        f"Optimize pricing and package deals for {user_type} preferences",
                        f"Implement {user_type}-specific quick responses"
                    ],
                    'expected_impact': f"Improve conversion rate for {user_type} users by 15-20%"
                })

        # Channel personalization
        for channel in self.channel_analysis.index:
            strategies.append({
                'segment': f"{channel} Users",
                'strategy': f"Optimize user experience for {channel} platform",
                'key_actions': [
                    f"Develop {channel}-specific UI components",
                    f"Implement platform-optimized response templates",
                    f"Create {channel}-exclusive offers and features"
                ],
                'expected_impact': f"Increase {channel} engagement by 10-15%"
            })

        # Geographic personalization
        top_locations = self.location_analysis.head(3).index
        for location in top_locations:
            strategies.append({
                'segment': f"{location} Based Users",
                'strategy': f"Localize travel recommendations for {location} market",
                'key_actions': [
                    f"Add {location}-specific destinations and travel tips",
                    f"Implement local payment methods for {location}",
                    f"Create seasonal offers based on {location} travel patterns"
                ],
                'expected_impact': f"Improve {location} market conversion by 12-18%"
            })

        return strategies

# %%
# Generate comprehensive recommendations
print("💡 Generating comprehensive optimization recommendations...")
complete_recommender = CompleteOptimizationRecommender(
    df, basic_metrics, intent_analysis, sentiment_df,
    channel_analysis, location_analysis, login_analysis, language_analysis
)

comprehensive_recommendations = complete_recommender.generate_comprehensive_recommendations()
personalization_strategies = complete_recommender.generate_personalization_strategies()

print("\n" + "="*80)
print("🚀 COMPREHENSIVE OPTIMIZATION RECOMMENDATIONS")
print("="*80)

for i, rec in enumerate(comprehensive_recommendations, 1):
    print(f"\n{i}. [{rec['priority']} Priority] {rec['category']}")
    print(f"   📊 Issue: {rec['issue']}")
    print(f"   🎯 Recommendation: {rec['recommendation']}")
    print(f"   ⚡ Impact: {rec['impact']}")

print("\n" + "="*80)
print("🎯 PERSONALIZATION STRATEGIES")
print("="*80)

for i, strategy in enumerate(personalization_strategies, 1):
    print(f"\n{i}. Target Segment: {strategy['segment']}")
    print(f"   🎯 Strategy: {strategy['strategy']}")
    print(f"   ⚡ Key Actions:")
    for action in strategy['key_actions']:
        print(f"      • {action}")
    print(f"   📈 Expected Impact: {strategy['expected_impact']}")

# %%
# %% [markdown]
# ## 8. COMPREHENSIVE ANALYTICS REPORT EXPORT

# %%
def generate_complete_analytics_report(metrics, intent_analysis, recommendations, strategies,
                                     channel_analysis, location_analysis, login_analysis, language_analysis):
    """Generate comprehensive analytics report with ALL features"""
    report = {
        "report_metadata": {
            "generated_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "time_period": "Last 30 days",
            "total_conversations": metrics['total_conversations'],
            "total_messages": metrics['total_messages'],
            "data_scope": "Complete segmentation analysis"
        },
        "performance_summary": {
            "success_rate": round(metrics['success_rate'], 2),
            "average_rating": round(metrics['avg_user_rating'], 2),
            "conversation_duration": round(metrics['avg_conversation_duration_min'], 2),
            "user_satisfaction": "High" if metrics['avg_user_rating'] > 4.0 else "Medium" if metrics['avg_user_rating'] > 3.0 else "Low"
        },
        "segmentation_insights": {
            "top_performing_channel": channel_analysis['booking_success_rate'].idxmax(),
            "top_location": location_analysis['total_conversations'].idxmax(),
            "login_status_gap": round(login_analysis.loc['Logged-in', 'booking_success_rate'] - login_analysis.loc['Guest', 'booking_success_rate'], 1),
            "preferred_language": language_analysis['total_conversations'].idxmax()
        },
        "key_findings": {
            "top_intents": intent_analysis.head(3).index.tolist(),
            "most_active_user_type": user_behavior['total_conversations'].idxmax(),
            "conversation_complexity": "High" if metrics['messages_per_conversation'] > 8 else "Medium" if metrics['messages_per_conversation'] > 5 else "Low",
            "channel_distribution": channel_analysis['total_conversations'].to_dict(),
            "geographic_coverage": len(location_analysis)
        },
        "optimization_recommendations": recommendations,
        "personalization_strategies": strategies,
        "next_steps": [
            "Implement A/B testing for channel-specific optimizations",
            "Develop localized content for underperforming geographic markets",
            "Create guest-to-logged-in conversion funnel",
            "Enhance multilingual support based on language performance",
            "Monitor sentiment trends and implement proactive support routing"
        ]
    }

    return report

# Generate final comprehensive report
complete_analytics_report = generate_complete_analytics_report(
    basic_metrics, intent_analysis, comprehensive_recommendations, personalization_strategies,
    channel_analysis, location_analysis, login_analysis, language_analysis
)

print("📄 COMPREHENSIVE ANALYTICS REPORT GENERATED SUCCESSFULLY!")
print("\n" + "="*60)
print("KEY PERFORMANCE INDICATORS:")
print("="*60)
for key, value in complete_analytics_report["performance_summary"].items():
    print(f"{key.replace('_', ' ').title()}: {value}")

print(f"\n🎯 SEGMENTATION INSIGHTS:")
print(f"• Top Performing Channel: {complete_analytics_report['segmentation_insights']['top_performing_channel']}")
print(f"• Most Active Location: {complete_analytics_report['segmentation_insights']['top_location']}")
print(f"• Login Status Performance Gap: {complete_analytics_report['segmentation_insights']['login_status_gap']}%")
print(f"• Preferred Language: {complete_analytics_report['segmentation_insights']['preferred_language']}")

# Save report to JSON
with open('complete_chatbot_analytics_report.json', 'w') as f:
    json.dump(complete_analytics_report, f, indent=2)

print("\n💾 Complete report saved as 'complete_chatbot_analytics_report.json'")

# %%
# %% [markdown]
# ## 🎉 COMPLETE IMPLEMENTATION SUMMARY

print("\n" + "="*80)
print("✅ TRAVEL CHATBOT ANALYTICS - COMPLETE IMPLEMENTATION SUCCESS!")
print("="*80)

print("\n🎯 ALL FEATURES INTEGRATED:")
print("✓ Complete Analytics Report")

print(f"\n📊 ANALYTICS GENERATED:")
print(f"• {basic_metrics['total_conversations']} conversations analyzed")
print(f"• {len(intent_analysis)} different intents identified")
print(f"• {len(channel_analysis)} channels performance evaluated")
print(f"• {len(location_analysis)} geographic locations analyzed")
print(f"• {len(language_analysis)} language preferences tracked")
print(f"• {len(comprehensive_recommendations)} optimization recommendations")
print(f"• {len(personalization_strategies)} personalization strategies")

print(f"\n🚀 KEY INSIGHTS:")
print(f"• Overall Success Rate: {basic_metrics['success_rate']:.1f}%")
print(f"• Best Performing Channel: {channel_analysis['booking_success_rate'].idxmax()}")
print(f"• Most Active User Type: {user_behavior['total_conversations'].idxmax()}")
print(f"• Geographic Coverage: {len(location_analysis)} countries")
print(f"• Sentiment Distribution: {sentiment_df['sentiment_label'].value_counts().to_dict()}")


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


🔄 Generating complete travel chatbot data with ALL segmentation features...
✅ Generated 3979 messages from 500 conversations
📊 Complete dataset shape: (3979, 15)

🎯 ALL SEGMENTATION COLUMNS AVAILABLE:
• User Types: ['business' 'family' 'couple' 'backpacker']
• Budget Categories: ['high' 'medium' 'low']
• Channels: ['WhatsApp' 'Mobile_App' 'Facebook_Messenger' 'Web']
• Locations: ['Australia' 'USA' 'UK' 'Japan' 'Brazil' 'Germany' 'India' 'France']
• Login Status: [ True False]
• Languages: ['Hindi' 'Spanish' 'French' 'German' 'English' 'Japanese']
📈 Calculating basic chatbot performance metrics...

🤖 BASIC CHATBOT PERFORMANCE METRICS
Total Conversations: 500
Total Messages: 3979
User Messages: 2593
Bot Messages: 1386
Messages Per Conversation: 7.958
Success Rate: 47.40%
Avg Conversation Duration Min: 18.87 min
Median Conversation Duration Min: 19.00 min
Avg User Rating: 3.10%
Rating Response Rate: 576.80%
🔍 Performing advanced NLP analysis...
✅ Advanced NLP analysis completed!

😊 SENTIM

🔮 Performing topic modeling...

📚 DISCOVERED CONVERSATION TOPICS
Topic_0: info, great, thanks, need, want
Topic_1: flight, options, month, book, want
Topic_2: think, ll, helps, thanks, flights
Topic_3: trip, planning, hi, people, booking
Topic_4: travel, help, need, suggestions, hey
💡 Generating comprehensive optimization recommendations...

🚀 COMPREHENSIVE OPTIMIZATION RECOMMENDATIONS

1. [High Priority] Success Rate
   📊 Issue: Low booking success rate (47.4%)
   🎯 Recommendation: Implement better intent recognition and provide more personalized options
   ⚡ Impact: Direct revenue impact

2. [High Priority] Intent Handling
   📊 Issue: Poor success rate for 'closing' intent (47.4%)
   🎯 Recommendation: Improve training data and response accuracy for closing requests
   ⚡ Impact: User satisfaction

3. [High Priority] Intent Handling
   📊 Issue: Poor success rate for 'greeting' intent (47.4%)
   🎯 Recommendation: Improve training data and response accuracy for greeting requests
   ⚡ Imp

# Task 4: Critical Evaluation and Testing Strategy

In [ ]:
# =============================================
# INSTALLATION & IMPORTS FOR INTERACTIVE PLOTS
# =============================================

!pip install plotly nbformat

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
from scipy import stats
from scipy.spatial.distance import jensenshannon
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')

print("📊 Loading Interactive Plotly Dashboard...")

📊 Loading Interactive Plotly Dashboard...


In [ ]:
# =============================================
# SECTION 1: INTERACTIVE A/B TESTING DASHBOARD
# =============================================

class InteractiveABTesting:
    """Interactive A/B Testing with Plotly"""

    def __init__(self):
        self.banking_intents = [
            'balance_inquiry', 'transfer_funds', 'loan_application',
            'card_issues', 'bill_payment', 'account_services', 'fraud_alert'
        ]

    def create_ab_testing_dashboard(self, df, results):
        """Create interactive A/B testing dashboard"""

        # Create subplot layout
        fig = make_subplots(
            rows=2, cols=3,
            subplot_titles=[
                'Conversion Rate Comparison',
                'Fallback Rate Comparison',
                'Satisfaction Score Comparison',
                'Session Length Distribution',
                'Resolution Time Distribution',
                'Statistical Significance Summary'
            ],
            specs=[
                [{"type": "bar"}, {"type": "bar"}, {"type": "bar"}],
                [{"type": "box"}, {"type": "box"}, {"type": "pie"}]
            ],
            vertical_spacing=0.12,
            horizontal_spacing=0.08
        )

        # 1. Conversion Rates
        conversion_rates = df.groupby('version')['conversion'].mean()
        fig.add_trace(
            go.Bar(
                x=['Version A (Traditional)', 'Version B (LLM-enhanced)'],
                y=conversion_rates.values,
                marker_color=['lightcoral', 'lightsteelblue'],
                text=[f'{val:.3f}' for val in conversion_rates.values],
                textposition='auto',
                name='Conversion Rate'
            ), row=1, col=1
        )

        # 2. Fallback Rates
        fallback_rates = df.groupby('version')['fallback_occurred'].mean()
        fig.add_trace(
            go.Bar(
                x=['Version A (Traditional)', 'Version B (LLM-enhanced)'],
                y=fallback_rates.values,
                marker_color=['lightcoral', 'lightsteelblue'],
                text=[f'{val:.3f}' for val in fallback_rates.values],
                textposition='auto',
                name='Fallback Rate'
            ), row=1, col=2
        )

        # 3. Satisfaction Scores
        satisfaction_means = df.groupby('version')['satisfaction_score'].mean()
        fig.add_trace(
            go.Bar(
                x=['Version A (Traditional)', 'Version B (LLM-enhanced)'],
                y=satisfaction_means.values,
                marker_color=['lightcoral', 'lightsteelblue'],
                text=[f'{val:.2f}' for val in satisfaction_means.values],
                textposition='auto',
                name='Satisfaction Score'
            ), row=1, col=3
        )

        # 4. Session Turns Distribution
        version_a_turns = df[df['version'] == 'A']['session_turns']
        version_b_turns = df[df['version'] == 'B']['session_turns']

        fig.add_trace(
            go.Box(
                y=version_a_turns,
                name='Version A (Traditional)',
                marker_color='lightcoral',
                boxpoints='outliers'
            ), row=2, col=1
        )

        fig.add_trace(
            go.Box(
                y=version_b_turns,
                name='Version B (LLM-enhanced)',
                marker_color='lightsteelblue',
                boxpoints='outliers'
            ), row=2, col=1
        )

        # 5. Resolution Time Distribution
        version_a_time = df[df['version'] == 'A']['resolution_time_sec']
        version_b_time = df[df['version'] == 'B']['resolution_time_sec']

        fig.add_trace(
            go.Box(
                y=version_a_time,
                name='Version A (Traditional)',
                marker_color='lightcoral',
                boxpoints='outliers',
                showlegend=False
            ), row=2, col=2
        )

        fig.add_trace(
            go.Box(
                y=version_b_time,
                name='Version B (LLM-enhanced)',
                marker_color='lightsteelblue',
                boxpoints='outliers',
                showlegend=False
            ), row=2, col=2
        )

        # 6. Statistical Significance Pie Chart
        significant_count = sum(1 for result in results.values() if result['significant'])
        total_tests = len(results)
        labels = ['Significant', 'Not Significant']
        values = [significant_count, total_tests - significant_count]

        fig.add_trace(
            go.Pie(
                labels=labels,
                values=values,
                marker_colors=['lightgreen', 'lightcoral'],
                textinfo='percent+label',
                hoverinfo='label+value',
                name='Significance Summary'
            ), row=2, col=3
        )

        # Update layout
        fig.update_layout(
            title_text="🏦 Banking Chatbot A/B Testing Dashboard<br><sub>Interactive Analysis: Traditional vs LLM-enhanced Responses</sub>",
            height=800,
            showlegend=True,
            template="plotly_white",
            title_x=0.5,
            font=dict(size=12)
        )

        # Update y-axis labels
        fig.update_yaxes(title_text="Rate", row=1, col=1)
        fig.update_yaxes(title_text="Rate", row=1, col=2)
        fig.update_yaxes(title_text="Score (1-5)", row=1, col=3)
        fig.update_yaxes(title_text="Session Turns", row=2, col=1)
        fig.update_yaxes(title_text="Seconds", row=2, col=2)

        fig.show()

        # Create detailed statistical results table
        self.create_statistical_table(results)

    def create_statistical_table(self, results):
        """Create interactive statistical results table"""

        table_data = []
        for test_name, result in results.items():
            improvement = ""
            if 'rate' in result:
                improvement = f"{((result['version_b_rate'] - result['version_a_rate'])/result['version_a_rate']*100):+.1f}%"
            elif 'mean' in result:
                improvement = f"{((result['version_b_mean'] - result['version_a_mean'])/result['version_a_mean']*100):+.1f}%"

            table_data.append({
                'Test Type': result['test'],
                'Metric': result['metric'],
                'P-Value': f"{result['p_value']:.4f}",
                'Significance': '✅ SIGNIFICANT' if result['significant'] else '❌ NOT SIGNIFICANT',
                'Improvement': improvement,
                'Version A': f"{result.get('version_a_rate', result.get('version_a_mean', 0)):.3f}",
                'Version B': f"{result.get('version_b_rate', result.get('version_b_mean', 0)):.3f}"
            })

        df_table = pd.DataFrame(table_data)

        # Create interactive table
        fig = go.Figure(data=[go.Table(
            header=dict(
                values=list(df_table.columns),
                fill_color='lightsteelblue',
                align='left',
                font=dict(size=14, color='white')
            ),
            cells=dict(
                values=[df_table[col] for col in df_table.columns],
                fill_color=['white', 'lightyellow'][::2],
                align='left',
                font=dict(size=12)
            )
        )])

        fig.update_layout(
            title="📊 Detailed Statistical Test Results",
            height=400,
            title_x=0.5
        )

        fig.show()

# Generate sample data and create dashboard
print("📊 SECTION 1: INTERACTIVE A/B TESTING DASHBOARD")
print("=" * 50)

# Generate sample data (reusing the previous data generation)
def simulate_banking_ab_data(total_users=2000):
    np.random.seed(42)
    banking_intents = ['balance_inquiry', 'transfer_funds', 'loan_application', 'card_issues', 'bill_payment', 'account_services', 'fraud_alert']

    data = []
    for user_id in range(total_users):
        version = np.random.choice(['A', 'B'], p=[0.5, 0.5])

        if version == 'A':  # Traditional
            conversion_prob = 0.18
            satisfaction = np.random.normal(3.7, 0.9)
            session_turns = np.random.poisson(6)
            fallback_prob = 0.15
            resolution_time = np.random.normal(45, 15)
        else:  # Version B
            conversion_prob = 0.25
            satisfaction = np.random.normal(4.2, 0.7)
            session_turns = np.random.poisson(8)
            fallback_prob = 0.09
            resolution_time = np.random.normal(35, 12)

        conversion = np.random.binomial(1, conversion_prob)
        satisfaction = max(1, min(5, satisfaction))
        session_turns = max(1, session_turns)
        fallback = np.random.binomial(1, fallback_prob)
        resolution_time = max(10, resolution_time)
        primary_intent = np.random.choice(banking_intents, p=[0.25, 0.20, 0.15, 0.15, 0.10, 0.10, 0.05])

        data.append({
            'user_id': f"user_{user_id}",
            'version': version,
            'conversion': conversion,
            'satisfaction_score': satisfaction,
            'session_turns': session_turns,
            'fallback_occurred': fallback,
            'resolution_time_sec': resolution_time,
            'primary_intent': primary_intent
        })

    return pd.DataFrame(data)

# Create dashboard
ab_dashboard = InteractiveABTesting()
banking_df = simulate_banking_ab_data(2000)

# Run statistical tests (simplified)
group_a = banking_df[banking_df['version'] == 'A']
group_b = banking_df[banking_df['version'] == 'B']

results = {}
# Conversion rate chi-square
conversion_contingency = pd.crosstab(banking_df['version'], banking_df['conversion'])
chi2_conv, p_conv, _, _ = stats.chi2_contingency(conversion_contingency)
results['conversion'] = {
    'test': 'Chi-square', 'metric': 'Conversion Rate', 'p_value': p_conv,
    'significant': p_conv < 0.05, 'version_a_rate': group_a['conversion'].mean(),
    'version_b_rate': group_b['conversion'].mean()
}

# Fallback rate chi-square
fallback_contingency = pd.crosstab(banking_df['version'], banking_df['fallback_occurred'])
chi2_fallback, p_fallback, _, _ = stats.chi2_contingency(fallback_contingency)
results['fallback'] = {
    'test': 'Chi-square', 'metric': 'Fallback Rate', 'p_value': p_fallback,
    'significant': p_fallback < 0.05, 'version_a_rate': group_a['fallback_occurred'].mean(),
    'version_b_rate': group_b['fallback_occurred'].mean()
}

# Satisfaction t-test
t_stat_sat, p_sat = stats.ttest_ind(group_a['satisfaction_score'], group_b['satisfaction_score'])
results['satisfaction'] = {
    'test': 'T-test', 'metric': 'Satisfaction Score', 'p_value': p_sat,
    'significant': p_sat < 0.05, 'version_a_mean': group_a['satisfaction_score'].mean(),
    'version_b_mean': group_b['satisfaction_score'].mean()
}

# Create interactive dashboard
ab_dashboard.create_ab_testing_dashboard(banking_df, results)

📊 SECTION 1: INTERACTIVE A/B TESTING DASHBOARD


In [ ]:
# =============================================
# SECTION 2: INTERACTIVE INTENT DRIFT DASHBOARD
# =============================================

class InteractiveDriftDashboard:
    """Interactive Intent Drift Detection with Plotly"""

    def __init__(self):
        self.banking_intents = [
            'balance_inquiry', 'transfer_funds', 'loan_application',
            'card_issues', 'bill_payment', 'account_services', 'fraud_alert'
        ]

    def create_drift_dashboard(self, temporal_data, drift_results):
        """Create interactive drift detection dashboard"""

        # Create subplot layout
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=[
                'Intent Distribution Drift (JSD)',
                'Confidence Distribution Drift (PSI)',
                'Key Intent Distribution Over Time',
                'Confidence vs Fallback Rate Trends'
            ],
            specs=[
                [{"type": "scatter"}, {"type": "scatter"}],
                [{"type": "scatter"}, {"type": "scatter"}]
            ],
            vertical_spacing=0.12
        )

        # 1. JSD Drift Detection
        fig.add_trace(
            go.Scatter(
                x=drift_results['day'],
                y=drift_results['jsd_distance'],
                mode='lines+markers',
                name='JSD Distance',
                line=dict(color='blue', width=3),
                marker=dict(size=6)
            ), row=1, col=1
        )

        # Add drift threshold
        fig.add_trace(
            go.Scatter(
                x=drift_results['day'],
                y=[0.1] * len(drift_results),
                mode='lines',
                name='Drift Threshold (0.1)',
                line=dict(color='red', width=2, dash='dash'),
                showlegend=True
            ), row=1, col=1
        )

        # Highlight drift areas
        drift_days = drift_results[drift_results['jsd_distance'] > 0.1]
        if not drift_days.empty:
            fig.add_trace(
                go.Scatter(
                    x=drift_days['day'],
                    y=drift_days['jsd_distance'],
                    mode='markers',
                    name='Drift Detected',
                    marker=dict(color='red', size=8, symbol='circle'),
                    showlegend=True
                ), row=1, col=1
            )

        # 2. PSI Drift Detection
        fig.add_trace(
            go.Scatter(
                x=drift_results['day'],
                y=drift_results['psi_index'],
                mode='lines+markers',
                name='PSI Index',
                line=dict(color='green', width=3),
                marker=dict(size=6, symbol='square')
            ), row=1, col=2
        )

        # Add PSI threshold
        fig.add_trace(
            go.Scatter(
                x=drift_results['day'],
                y=[0.1] * len(drift_results),
                mode='lines',
                name='PSI Threshold (0.1)',
                line=dict(color='red', width=2, dash='dash'),
                showlegend=False
            ), row=1, col=2
        )

        # 3. Key Intent Distribution Over Time
        daily_intents = temporal_data.groupby(['day', 'predicted_intent']).size().unstack(fill_value=0)
        daily_intents_pct = daily_intents.div(daily_intents.sum(axis=1), axis=0)

        # Plot top 4 intents for clarity
        top_intents = ['balance_inquiry', 'transfer_funds', 'loan_application', 'card_issues']
        for intent in top_intents:
            if intent in daily_intents_pct.columns:
                fig.add_trace(
                    go.Scatter(
                        x=daily_intents_pct.index,
                        y=daily_intents_pct[intent],
                        mode='lines',
                        name=intent.replace('_', ' ').title(),
                        line=dict(width=3)
                    ), row=2, col=1
                )

        # 4. Confidence vs Fallback Rate Trends
        daily_metrics = temporal_data.groupby('day').agg({
            'intent_confidence': 'mean',
            'is_fallback': 'mean'
        }).reset_index()

        # Confidence scores
        fig.add_trace(
            go.Scatter(
                x=daily_metrics['day'],
                y=daily_metrics['intent_confidence'],
                mode='lines',
                name='Avg Confidence',
                line=dict(color='blue', width=3),
                yaxis='y'
            ), row=2, col=2
        )

        # Fallback rates (secondary y-axis)
        fig.add_trace(
            go.Scatter(
                x=daily_metrics['day'],
                y=daily_metrics['is_fallback'] * 100,
                mode='lines',
                name='Fallback Rate %',
                line=dict(color='red', width=3),
                yaxis='y2'
            ), row=2, col=2
        )

        # Update layout for dual y-axis
        fig.update_layout(
            yaxis2=dict(
                title="Fallback Rate (%)",
                titlefont=dict(color="red"),
                tickfont=dict(color="red"),
                overlaying="y",
                side="right"
            )
        )

        # Update overall layout
        fig.update_layout(
            title_text="🔄 Banking Chatbot Intent Drift Detection Dashboard<br><sub>Interactive Monitoring of Intent Distribution & Confidence Stability</sub>",
            height=800,
            showlegend=True,
            template="plotly_white",
            title_x=0.5,
            font=dict(size=12)
        )

        # Update axis labels
        fig.update_xaxes(title_text="Day", row=2, col=1)
        fig.update_xaxes(title_text="Day", row=2, col=2)
        fig.update_yaxes(title_text="JSD Distance", row=1, col=1)
        fig.update_yaxes(title_text="PSI Index", row=1, col=2)
        fig.update_yaxes(title_text="Proportion", row=2, col=1)
        fig.update_yaxes(title_text="Average Confidence", row=2, col=2)

        fig.show()

        # Create summary metrics
        self.create_drift_summary(drift_results)

    def create_drift_summary(self, drift_results):
        """Create drift detection summary metrics"""

        jsd_drift_days = len(drift_results[drift_results['jsd_drift_detected']])
        psi_drift_days = len(drift_results[drift_results['psi_drift_detected']])

        metrics_data = [
            ['JSD Drift Days', f"{jsd_drift_days}/{len(drift_results)}", f"{(jsd_drift_days/len(drift_results)*100):.1f}%"],
            ['PSI Drift Days', f"{psi_drift_days}/{len(drift_results)}", f"{(psi_drift_days/len(drift_results)*100):.1f}%"],
            ['Max JSD', f"{drift_results['jsd_distance'].max():.3f}", "High" if drift_results['jsd_distance'].max() > 0.15 else "Medium"],
            ['Max PSI', f"{drift_results['psi_index'].max():.3f}", "High" if drift_results['psi_index'].max() > 0.25 else "Medium"]
        ]

        fig = go.Figure(data=[go.Table(
            header=dict(
                values=['Metric', 'Value', 'Assessment'],
                fill_color='lightsteelblue',
                align='center',
                font=dict(size=14, color='white')
            ),
            cells=dict(
                values=[[row[0] for row in metrics_data],
                        [row[1] for row in metrics_data],
                        [row[2] for row in metrics_data]],
                fill_color=[['white', 'lightyellow'] * 2],
                align='center',
                font=dict(size=12)
            )
        )])

        fig.update_layout(
            title="📈 Drift Detection Summary Metrics",
            height=300,
            title_x=0.5
        )

        fig.show()

# Generate sample temporal data
def simulate_temporal_banking_data(days=60, sessions_per_day=80):
    np.random.seed(42)
    banking_intents = ['balance_inquiry', 'transfer_funds', 'loan_application', 'card_issues', 'bill_payment', 'account_services', 'fraud_alert']

    data = []
    for day in range(days):
        if day < 20:
            intent_probs = [0.25, 0.20, 0.15, 0.15, 0.10, 0.10, 0.05]
            confidence_mean = 0.85
        elif day < 40:
            intent_probs = [0.20, 0.18, 0.25, 0.15, 0.08, 0.09, 0.05]
            confidence_mean = 0.82
        else:
            intent_probs = [0.22, 0.15, 0.12, 0.25, 0.08, 0.08, 0.10]
            confidence_mean = 0.78

        for session in range(sessions_per_day):
            intent = np.random.choice(banking_intents, p=intent_probs)
            confidence = np.random.beta(confidence_mean * 10, (1 - confidence_mean) * 10)
            confidence = min(0.99, max(0.1, confidence))
            is_fallback = 1 if confidence < 0.3 else 0

            data.append({
                'day': day,
                'session_id': f"session_{day}_{session}",
                'predicted_intent': intent,
                'intent_confidence': confidence,
                'is_fallback': is_fallback,
                'session_turns': np.random.poisson(7)
            })

    return pd.DataFrame(data)

# Calculate JSD and PSI (simplified)
def calculate_drift_metrics(temporal_data, banking_intents):
    baseline_data = temporal_data[temporal_data['day'] < 10]
    baseline_intent_dist = baseline_data['predicted_intent'].value_counts(normalize=True)
    baseline_intent_dist = baseline_intent_dist.reindex(banking_intents, fill_value=0)
    baseline_confidence = baseline_data['intent_confidence']

    drift_metrics = []
    for day in temporal_data['day'].unique():
        if day < 10:
            continue
        current_data = temporal_data[temporal_data['day'] == day]
        if len(current_data) == 0:
            continue

        # JSD calculation
        current_intent_dist = current_data['predicted_intent'].value_counts(normalize=True)
        current_intent_dist = current_intent_dist.reindex(banking_intents, fill_value=0)

        # Simplified JSD
        m = 0.5 * (baseline_intent_dist.values + current_intent_dist.values)
        jsd = 0.5 * (stats.entropy(baseline_intent_dist.values, m) + stats.entropy(current_intent_dist.values, m))

        # Simplified PSI
        current_confidence = current_data['intent_confidence']
        psi = 0.05 + np.random.random() * 0.2  # Placeholder

        drift_metrics.append({
            'day': day,
            'jsd_distance': jsd,
            'psi_index': psi,
            'jsd_drift_detected': jsd > 0.1,
            'psi_drift_detected': psi > 0.1
        })

    return pd.DataFrame(drift_metrics)

print("\n🔄 SECTION 2: INTERACTIVE INTENT DRIFT DASHBOARD")
print("=" * 50)

# Create drift dashboard
drift_dashboard = InteractiveDriftDashboard()
temporal_df = simulate_temporal_banking_data(60, 80)
drift_metrics_df = calculate_drift_metrics(temporal_df, drift_dashboard.banking_intents)
drift_dashboard.create_drift_dashboard(temporal_df, drift_metrics_df)


🔄 SECTION 2: INTERACTIVE INTENT DRIFT DASHBOARD


In [ ]:
# =============================================
# SECTION 3: INTERACTIVE ANOMALY DETECTION DASHBOARD
# =============================================

class InteractiveAnomalyDashboard:
    """Interactive Anomaly Detection with Plotly"""

    def create_anomaly_dashboard(self, temporal_data, session_anomalies, change_points):
        """Create interactive anomaly detection dashboard"""

        # Create subplot layout
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=[
                'Session Anomaly Score Distribution',
                'Daily Anomaly Detection Rate',
                'Change Point Detection in Fallback Rate',
                'Normal vs Anomalous Session Characteristics'
            ],
            specs=[
                [{"type": "histogram"}, {"type": "bar"}],
                [{"type": "scatter"}, {"type": "bar"}]
            ],
            vertical_spacing=0.12
        )

        # 1. Anomaly Score Distribution
        normal_scores = session_anomalies[~session_anomalies['is_anomaly']]['anomaly_score']
        anomalous_scores = session_anomalies[session_anomalies['is_anomaly']]['anomaly_score']

        fig.add_trace(
            go.Histogram(
                x=normal_scores,
                name='Normal Sessions',
                marker_color='green',
                opacity=0.7,
                nbinsx=30
            ), row=1, col=1
        )

        fig.add_trace(
            go.Histogram(
                x=anomalous_scores,
                name='Anomalous Sessions',
                marker_color='red',
                opacity=0.7,
                nbinsx=15
            ), row=1, col=1
        )

        # 2. Daily Anomaly Rate
        daily_anomalies = session_anomalies.groupby('day')['is_anomaly'].mean() * 100

        fig.add_trace(
            go.Bar(
                x=daily_anomalies.index,
                y=daily_anomalies.values,
                name='Anomaly Rate',
                marker_color='red',
                opacity=0.7
            ), row=1, col=2
        )

        # Add change point markers
        for _, cp in change_points.iterrows():
            fig.add_trace(
                go.Scatter(
                    x=[cp['day']],
                    y=[daily_anomalies.loc[cp['day']] if cp['day'] in daily_anomalies.index else 0],
                    mode='markers+text',
                    marker=dict(color='darkred', size=12, symbol='star'),
                    text=[f'Day {cp["day"]}'],
                    textposition='top center',
                    name='Change Point',
                    showlegend=False
                ), row=1, col=2
            )

        # 3. Change Point Detection
        daily_fallback = temporal_data.groupby('day')['is_fallback'].mean() * 100

        fig.add_trace(
            go.Scatter(
                x=daily_fallback.index,
                y=daily_fallback.values,
                mode='lines',
                name='Fallback Rate',
                line=dict(color='orange', width=3)
            ), row=2, col=1
        )

        # Highlight change points
        for _, cp in change_points.iterrows():
            fig.add_trace(
                go.Scatter(
                    x=[cp['day'], cp['day']],
                    y=[0, daily_fallback.max()],
                    mode='lines',
                    line=dict(color='red', width=2, dash='dash'),
                    name=f'Change Point Day {cp["day"]}',
                    showlegend=False
                ), row=2, col=1
            )

            fig.add_annotation(
                x=cp['day'],
                y=daily_fallback.max() * 0.9,
                text=f"Day {cp['day']}",
                showarrow=True,
                arrowhead=2,
                arrowsize=1,
                arrowwidth=2,
                arrowcolor='red',
                row=2, col=1
            )

        # 4. Session Characteristics Comparison
        normal_sessions = session_anomalies[~session_anomalies['is_anomaly']]
        anomalous_sessions = session_anomalies[session_anomalies['is_anomaly']]

        characteristics = ['session_turns', 'intent_confidence', 'is_fallback', 'unique_intents']
        char_labels = ['Session Turns', 'Confidence', 'Fallback Rate', 'Unique Intents']

        normal_means = [normal_sessions[char].mean() for char in characteristics]
        anomalous_means = [anomalous_sessions[char].mean() for char in characteristics]

        # Normalize for comparison
        max_vals = [max(n, a) for n, a in zip(normal_means, anomalous_means)]
        normal_norm = [n/max_vals[i] for i, n in enumerate(normal_means)]
        anomalous_norm = [a/max_vals[i] for i, a in enumerate(anomalous_means)]

        fig.add_trace(
            go.Bar(
                x=char_labels,
                y=normal_norm,
                name='Normal Sessions',
                marker_color='green',
                opacity=0.7
            ), row=2, col=2
        )

        fig.add_trace(
            go.Bar(
                x=char_labels,
                y=anomalous_norm,
                name='Anomalous Sessions',
                marker_color='red',
                opacity=0.7
            ), row=2, col=2
        )

        # Update layout
        fig.update_layout(
            title_text="🚨 Banking Chatbot Anomaly Detection Dashboard<br><sub>Interactive Monitoring of Unusual Patterns & Change Points</sub>",
            height=800,
            showlegend=True,
            template="plotly_white",
            title_x=0.5,
            font=dict(size=12),
            barmode='group'
        )

        # Update axis labels
        fig.update_xaxes(title_text="Anomaly Score", row=1, col=1)
        fig.update_xaxes(title_text="Day", row=1, col=2)
        fig.update_xaxes(title_text="Day", row=2, col=1)
        fig.update_xaxes(title_text="Session Characteristics", row=2, col=2)

        fig.update_yaxes(title_text="Density", row=1, col=1)
        fig.update_yaxes(title_text="Anomaly Rate (%)", row=1, col=2)
        fig.update_yaxes(title_text="Fallback Rate (%)", row=2, col=1)
        fig.update_yaxes(title_text="Normalized Value", row=2, col=2)

        fig.show()

        # Create anomaly summary
        self.create_anomaly_summary(session_anomalies, change_points)

    def create_anomaly_summary(self, session_anomalies, change_points):
        """Create anomaly detection summary"""

        total_anomalies = len(session_anomalies[session_anomalies['is_anomaly']])
        total_sessions = len(session_anomalies)

        summary_data = [
            ['Total Sessions', f"{total_sessions}", "-"],
            ['Anomalous Sessions', f"{total_anomalies}", f"{(total_anomalies/total_sessions*100):.1f}%"],
            ['Change Points', f"{len(change_points)}", "Detected"],
            ['Avg Anomaly Score', f"{session_anomalies['anomaly_score'].mean():.3f}", "Baseline"],
            ['Detection Rate', f"{(total_anomalies/total_sessions*100):.1f}%", "Optimal" if (total_anomalies/total_sessions*100) < 10 else "High"]
        ]

        # Characteristics comparison
        normal_sessions = session_anomalies[~session_anomalies['is_anomaly']]
        anomalous_sessions = session_anomalies[session_anomalies['is_anomaly']]

        if len(anomalous_sessions) > 0:
            char_comparison = [
                ['Fallback Rate',
                 f"{normal_sessions['is_fallback'].mean():.3f}",
                 f"{anomalous_sessions['is_fallback'].mean():.3f}"],
                ['Confidence',
                 f"{normal_sessions['intent_confidence'].mean():.3f}",
                 f"{anomalous_sessions['intent_confidence'].mean():.3f}"],
                ['Session Turns',
                 f"{normal_sessions['session_turns'].mean():.1f}",
                 f"{anomalous_sessions['session_turns'].mean():.1f}"]
            ]
        else:
            char_comparison = [['No anomalies detected', '-', '-']]

        # Create summary table
        fig = go.Figure(data=[go.Table(
            header=dict(
                values=['Metric', 'Value', 'Assessment'],
                fill_color='lightsteelblue',
                align='center',
                font=dict(size=14, color='white')
            ),
            cells=dict(
                values=[[row[0] for row in summary_data],
                        [row[1] for row in summary_data],
                        [row[2] for row in summary_data]],
                fill_color=[['white', 'lightyellow'] * 3],
                align='center',
                font=dict(size=12)
            )
        )])

        fig.update_layout(
            title="📊 Anomaly Detection Summary",
            height=350,
            title_x=0.5
        )

        fig.show()

        # Create characteristics comparison table
        if len(anomalous_sessions) > 0:
            fig2 = go.Figure(data=[go.Table(
                header=dict(
                    values=['Characteristic', 'Normal Sessions', 'Anomalous Sessions'],
                    fill_color='lightcoral',
                    align='center',
                    font=dict(size=14, color='white')
                ),
                cells=dict(
                    values=[[row[0] for row in char_comparison],
                            [row[1] for row in char_comparison],
                            [row[2] for row in char_comparison]],
                    fill_color=[['white', 'lightyellow'] * 2],
                    align='center',
                    font=dict(size=12)
                )
            )])

            fig2.update_layout(
                title="🔍 Session Characteristics Comparison",
                height=300,
                title_x=0.5
            )

            fig2.show()

print("\n🚨 SECTION 3: INTERACTIVE ANOMALY DETECTION DASHBOARD")
print("=" * 50)

# Generate sample anomaly data
def detect_anomalies(temporal_data):
    session_features = temporal_data.groupby('session_id').agg({
        'session_turns': 'max',
        'intent_confidence': 'mean',
        'is_fallback': 'sum',
        'day': 'first'
    }).reset_index()

    intent_diversity = temporal_data.groupby('session_id')['predicted_intent'].nunique().reset_index()
    intent_diversity.columns = ['session_id', 'unique_intents']
    session_features = session_features.merge(intent_diversity, on='session_id')

    feature_columns = ['session_turns', 'intent_confidence', 'is_fallback', 'unique_intents']
    X = session_features[feature_columns]

    iso_forest = IsolationForest(contamination=0.08, random_state=42)
    anomalies = iso_forest.fit_predict(X)
    anomaly_scores = iso_forest.decision_function(X)

    session_features['anomaly_score'] = -anomaly_scores
    session_features['is_anomaly'] = anomalies == -1

    return session_features

def simulate_change_points(temporal_data):
    daily_fallback = temporal_data.groupby('day')['is_fallback'].mean()
    change_points = []

    # Simulate some change points
    for day in [20, 40, 55]:
        change_points.append({
            'day': day,
            'metric': 'is_fallback',
            'p_value': 0.001,
            'change_magnitude': 0.1
        })

    return pd.DataFrame(change_points)

# Create anomaly dashboard
anomaly_dashboard = InteractiveAnomalyDashboard()
session_anomalies_df = detect_anomalies(temporal_df)
change_points_df = simulate_change_points(temporal_df)
anomaly_dashboard.create_anomaly_dashboard(temporal_df, session_anomalies_df, change_points_df)


🚨 SECTION 3: INTERACTIVE ANOMALY DETECTION DASHBOARD


In [ ]:
# =============================================
# COMPREHENSIVE INTERACTIVE DASHBOARD
# =============================================

class ComprehensiveBankingDashboard:
    """Comprehensive Interactive Dashboard combining all sections"""

    def create_executive_summary(self, ab_results, drift_results, anomaly_results):
        """Create executive summary dashboard"""

        # Calculate key metrics
        significant_ab_tests = sum(1 for result in ab_results.values() if result['significant'])
        jsd_drift_days = len(drift_results[drift_results['jsd_drift_detected']])
        total_anomalies = len(anomaly_results[anomaly_results['is_anomaly']])

        # Create summary cards
        fig = make_subplots(
            rows=2, cols=3,
            specs=[
                [{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}],
                [{"type": "bar", "colspan": 3}, None, None]
            ],
            vertical_spacing=0.15,
            subplot_titles=['', '', '', 'Overall Performance Assessment']
        )

        # 1. A/B Testing Success Rate
        fig.add_trace(
            go.Indicator(
                mode="gauge+number+delta",
                value=significant_ab_tests,
                delta={'reference': len(ab_results)/2, 'increasing': {'color': "green"}},
                gauge={
                    'axis': {'range': [None, len(ab_results)]},
                    'bar': {'color': "lightsteelblue"},
                    'steps': [
                        {'range': [0, len(ab_results)*0.5], 'color': "lightcoral"},
                        {'range': [len(ab_results)*0.5, len(ab_results)], 'color': "lightgreen"}
                    ]
                },
                title={'text': "A/B Tests Significant"},
                domain={'row': 0, 'column': 0}
            ), row=1, col=1
        )

        # 2. Drift Detection
        fig.add_trace(
            go.Indicator(
                mode="gauge+number",
                value=jsd_drift_days,
                gauge={
                    'axis': {'range': [0, len(drift_results)]},
                    'bar': {'color': "lightsteelblue"},
                    'steps': [
                        {'range': [0, len(drift_results)*0.3], 'color': "lightgreen"},
                        {'range': [len(drift_results)*0.3, len(drift_results)*0.7], 'color': "yellow"},
                        {'range': [len(drift_results)*0.7, len(drift_results)], 'color': "lightcoral"}
                    ]
                },
                title={'text': "Drift Days Detected"},
                domain={'row': 0, 'column': 1}
            ), row=1, col=2
        )

        # 3. Anomaly Detection
        fig.add_trace(
            go.Indicator(
                mode="gauge+number",
                value=total_anomalies,
                gauge={
                    'axis': {'range': [0, len(anomaly_results)*0.2]},
                    'bar': {'color': "lightsteelblue"},
                    'steps': [
                        {'range': [0, len(anomaly_results)*0.05], 'color': "lightgreen"},
                        {'range': [len(anomaly_results)*0.05, len(anomaly_results)*0.15], 'color': "yellow"},
                        {'range': [len(anomaly_results)*0.15, len(anomaly_results)*0.2], 'color': "lightcoral"}
                    ]
                },
                title={'text': "Anomalies Detected"},
                domain={'row': 0, 'column': 2}
            ), row=1, col=3
        )

        # 4. Performance Assessment Bar Chart
        categories = ['A/B Testing', 'Drift Detection', 'Anomaly Detection', 'Overall']
        scores = [
            (significant_ab_tests / len(ab_results)) * 100,
            (jsd_drift_days / len(drift_results)) * 100,
            min(100, (total_anomalies / len(anomaly_results)) * 500),  # Scale for better visualization
            ((significant_ab_tests / len(ab_results)) +
             (jsd_drift_days / len(drift_results)) +
             min(1, (total_anomalies / len(anomaly_results)) * 5)) / 3 * 100
        ]

        colors = ['lightsteelblue' if score >= 50 else 'lightcoral' for score in scores]

        fig.add_trace(
            go.Bar(
                x=categories,
                y=scores,
                marker_color=colors,
                text=[f'{score:.1f}%' for score in scores],
                textposition='auto',
            ), row=2, col=1
        )

        fig.update_layout(
            title_text="🏦 Banking Chatbot Comprehensive Evaluation Dashboard<br><sub>Executive Summary & Performance Overview</sub>",
            height=600,
            showlegend=False,
            template="plotly_white",
            title_x=0.5,
            font=dict(size=12)
        )

        fig.show()

        # Create recommendations table
        self.create_recommendations_table(significant_ab_tests, jsd_drift_days, total_anomalies, len(anomaly_results))

    def create_recommendations_table(self, ab_success, drift_days, anomalies, total_anomaly_sessions):
        """Create interactive recommendations table"""

        recommendations = []

        if ab_success >= 3:
            recommendations.append(["A/B Testing", "✅ EXCELLENT", "Continue LLM-enhanced version deployment"])
        elif ab_success >= 2:
            recommendations.append(["A/B Testing", "⚠️ GOOD", "Monitor LLM version with gradual rollout"])
        else:
            recommendations.append(["A/B Testing", "❌ NEEDS WORK", "Re-evaluate LLM enhancement strategy"])

        if drift_days > 0:
            recommendations.append(["Drift Detection", "✅ EFFECTIVE", "Maintain current monitoring frequency"])
        else:
            recommendations.append(["Drift Detection", "⚠️ MONITOR", "Increase monitoring sensitivity"])

        # Corrected condition: removed len() from 'anomalies'
        if 0 < anomalies < total_anomaly_sessions * 0.1:
            recommendations.append(["Anomaly Detection", "✅ OPTIMAL", "Current detection rate is effective"])
        elif anomalies >= total_anomaly_sessions * 0.1:
            recommendations.append(["Anomaly Detection", "⚠️ HIGH", "Investigate root causes of anomalies"])
        else:
            recommendations.append(["Anomaly Detection", "❌ INACTIVE", "Review detection thresholds"])


        recommendations.append(["Overall Strategy", "📊 COMPREHENSIVE", "Combine all three evaluation methods for continuous improvement"])

        fig = go.Figure(data=[go.Table(
            header=dict(
                values=['Component', 'Status', 'Recommendation'],
                fill_color='lightsteelblue',
                align='left',
                font=dict(size=14, color='white')
            ),
            cells=dict(
                values=[[row[0] for row in recommendations],
                        [row[1] for row in recommendations],
                        [row[2] for row in recommendations]],
                fill_color=[['white', 'lightyellow'] * 2],
                align='left',
                font=dict(size=12)
            )
        )])

        fig.update_layout(
            title="🎯 Strategic Recommendations & Action Plan",
            height=400,
            title_x=0.5
        )

        fig.show()

print("\n🏦 COMPREHENSIVE INTERACTIVE DASHBOARD")
print("=" * 50)

# Create comprehensive dashboard
comprehensive_dash = ComprehensiveBankingDashboard()

# Use previously generated data for summary
ab_success_count = sum(1 for result in results.values() if result['significant'])
jsd_drift_count = len(drift_metrics_df[drift_metrics_df['jsd_drift_detected']])
anomaly_count = len(session_anomalies_df[session_anomalies_df['is_anomaly']])

comprehensive_dash.create_executive_summary(
    results,
    drift_metrics_df,
    session_anomalies_df
)


🏦 COMPREHENSIVE INTERACTIVE DASHBOARD


# Task 5: Insightful Reporting and Visualization

In [ ]:
# =============================================
# FIXED ADVANCED BANKING CHATBOT DASHBOARD
# =============================================

import random
import uuid
import json
from datetime import datetime, timedelta
from collections import Counter, defaultdict
import math

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.ensemble import IsolationForest

random.seed(42)
np.random.seed(42)

# ----------------------------
# Configuration
# ----------------------------
NUM_SESSIONS = 2000  # Reduced for better performance
BASE_TIME = datetime.utcnow() - timedelta(days=30)
HUMAN_AGENT_HOURLY_COST = 25.0
AVG_HUMAN_CALL_DURATION_MIN = 7
DEVELOPMENT_COST = 5000.0
HOSTING_COST_PER_YEAR = 800.0
AVG_CONVERSION_VALUE = 50.0

INTENTS = [
    "greet", "check_balance", "transactions", "transfer_funds",
    "open_account", "loan_inquiry", "card_block", "faq_interest_rates",
    "update_details", "goodbye", "small_talk"
]

CONVERSION_INTENTS = {"transfer_funds", "open_account"}
CHANNELS = ["web", "mobile_app", "whatsapp", "facebook_messenger"]

# ----------------------------
# SIMPLIFIED DATA GENERATION
# ----------------------------

def generate_simplified_data(num_sessions=2000):
    """Generate simplified but realistic banking chatbot data"""

    sessions = []
    messages = []

    for i in range(num_sessions):
        session_id = f"session_{i}"
        user_id = f"user_{random.randint(1000, 99999)}"
        channel = random.choice(CHANNELS)
        day = i % 30

        # Create temporal patterns for drift simulation
        if day < 10:
            # Phase 1: Good performance
            resolution_prob = 0.78
            confidence_mean = 0.87
            fallback_prob = 0.10
        elif day < 20:
            # Phase 2: Slight degradation
            resolution_prob = 0.72
            confidence_mean = 0.82
            fallback_prob = 0.15
        else:
            # Phase 3: More degradation
            resolution_prob = 0.65
            confidence_mean = 0.75
            fallback_prob = 0.22

        # Generate session data
        session_data = {
            'session_id': session_id,
            'user_id': user_id,
            'channel': channel,
            'day': day,
            'timestamp': (BASE_TIME + timedelta(days=day)).isoformat(),
            'resolved': np.random.random() < resolution_prob,
            'used_fallback': np.random.random() < fallback_prob,
            'conversion': np.random.random() < 0.23,
            'post_rating': np.random.choice([1, 2, 3, 4, 5], p=[0.05, 0.1, 0.2, 0.4, 0.25]),
            'session_turns': max(2, int(np.random.poisson(5))),
            'true_intent': random.choice(INTENTS),
            'avg_confidence': np.random.normal(confidence_mean, 0.08),
            'response_time_ms': np.random.normal(450, 120)
        }

        # Add anomaly score (simplified)
        session_data['anomaly_score'] = np.random.exponential(0.5)
        if np.random.random() < 0.08:  # 8% anomalies
            session_data['anomaly_score'] = 2 + np.random.exponential(1)

        sessions.append(session_data)

        # Generate a few sample messages
        num_messages = session_data['session_turns']
        for turn in range(num_messages):
            messages.append({
                'session_id': session_id,
                'message_id': f"msg_{i}_{turn}",
                'turn': turn,
                'true_intent': session_data['true_intent'],
                'predicted_intent': session_data['true_intent'],
                'confidence': np.random.normal(confidence_mean, 0.1),
                'response_time_ms': np.random.normal(450, 120)
            })

    return pd.DataFrame(sessions), pd.DataFrame(messages)

print("🔄 Generating banking chatbot data...")
session_df, message_df = generate_simplified_data(2000)

print(f"✅ Generated {len(session_df)} sessions and {len(message_df)} messages")
print("Session data sample:")
display(session_df.head(2))

# =============================================
# FIXED ADVANCED DASHBOARD
# =============================================

class FixedAdvancedDashboard:
    """Fixed Advanced Banking Chatbot Dashboard"""

    def __init__(self, session_df, message_df):
        self.session_df = session_df
        self.message_df = message_df
        self.setup_colors()

    def setup_colors(self):
        """Setup color scheme"""
        self.colors = {
            'primary': '#1f77b4',
            'success': '#2ca02c',
            'warning': '#ff7f0e',
            'danger': '#d62728',
            'info': '#17becf',
            'secondary': '#7f7f7f'
        }

    def calculate_basic_metrics(self):
        """Calculate basic performance metrics"""
        total_sessions = len(self.session_df)
        resolution_rate = (self.session_df['resolved'].sum() / total_sessions) * 100
        conversion_rate = (self.session_df['conversion'].sum() / total_sessions) * 100
        fallback_rate = (self.session_df['used_fallback'].sum() / total_sessions) * 100
        avg_rating = self.session_df['post_rating'].mean()
        avg_confidence = self.session_df['avg_confidence'].mean() * 100

        return {
            'total_sessions': total_sessions,
            'resolution_rate': resolution_rate,
            'conversion_rate': conversion_rate,
            'fallback_rate': fallback_rate,
            'avg_rating': avg_rating,
            'avg_confidence': avg_confidence
        }

    def calculate_drift_metrics(self):
        """Calculate simple drift metrics"""
        daily_confidence = self.session_df.groupby('day')['avg_confidence'].mean()
        baseline = daily_confidence.iloc[:7].mean()  # First week as baseline
        current = daily_confidence.iloc[-1]
        drift_score = abs(current - baseline) * 100  # Convert to percentage

        return {
            'drift_score': drift_score,
            'status': 'HIGH' if drift_score > 15 else 'MODERATE' if drift_score > 8 else 'LOW'
        }

    def calculate_anomaly_metrics(self):
        """Calculate anomaly metrics"""
        # Simple anomaly detection based on multiple factors
        features = ['session_turns', 'avg_confidence', 'post_rating', 'response_time_ms']
        X = self.session_df[features]

        # Normalize
        X_normalized = (X - X.mean()) / X.std()

        # Simple threshold-based anomaly detection
        anomaly_scores = X_normalized.abs().max(axis=1)
        anomaly_threshold = 2.0
        anomalies_detected = (anomaly_scores > anomaly_threshold).sum()
        anomaly_rate = (anomalies_detected / len(self.session_df)) * 100

        return {
            'anomaly_rate': anomaly_rate,
            'anomalies_detected': anomalies_detected,
            'status': 'HIGH' if anomaly_rate > 10 else 'MODERATE' if anomaly_rate > 5 else 'LOW'
        }

    def create_executive_dashboard(self):
        """Create executive dashboard with KPIs"""

        metrics = self.calculate_basic_metrics()
        drift_metrics = self.calculate_drift_metrics()
        anomaly_metrics = self.calculate_anomaly_metrics()

        fig = make_subplots(
            rows=2, cols=3,
            specs=[
                [{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}],
                [{"type": "indicator"}, {"type": "indicator"}, {"type": "bar"}]
            ],
            subplot_titles=('', '', '', '', '', 'Channel Distribution')
        )

        # Main KPIs
        main_kpis = [
            (metrics['total_sessions'], "Total Sessions", "", 1, 1),
            (metrics['resolution_rate'], "Resolution Rate", "%", 1, 2),
            (metrics['conversion_rate'], "Conversion Rate", "%", 1, 3),
            (metrics['avg_rating'], "Avg Rating", "/5", 2, 1),
            (drift_metrics['drift_score'], "Drift Score", "%", 2, 2)
        ]

        for value, title, suffix, row, col in main_kpis:
            fig.add_trace(
                go.Indicator(
                    mode="number",
                    value=value,
                    number={'suffix': suffix, 'font': {'size': 24}},
                    title={'text': title, 'font': {'size': 14}},
                    domain={'row': row-1, 'column': col-1}
                ), row=row, col=col
            )

        # Channel distribution
        channel_counts = self.session_df['channel'].value_counts()
        fig.add_trace(
            go.Bar(
                x=channel_counts.index,
                y=channel_counts.values,
                marker_color=[self.colors['primary'], self.colors['success'],
                             self.colors['warning'], self.colors['danger']],
                text=channel_counts.values,
                textposition='auto'
            ), row=2, col=3
        )

        fig.update_layout(
            title_text="🏦 Banking Chatbot - Executive Dashboard",
            height=500,
            showlegend=False,
            template="plotly_white",
            title_x=0.5
        )

        return fig

    def create_performance_gauges(self):
        """Create performance gauges dashboard"""

        metrics = self.calculate_basic_metrics()
        drift_metrics = self.calculate_drift_metrics()
        anomaly_metrics = self.calculate_anomaly_metrics()

        fig = make_subplots(
            rows=2, cols=3,
            specs=[[{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}],
                   [{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}]],
            subplot_titles=('Resolution Rate', 'Conversion Rate', 'User Rating',
                           'Drift Score', 'Anomaly Rate', 'Confidence Score')
        )

        # Gauge configurations: (value, title, max_value, thresholds)
        gauge_configs = [
            (metrics['resolution_rate'], "Resolution Rate", 100, [70, 85]),
            (metrics['conversion_rate'], "Conversion Rate", 50, [15, 30]),
            (metrics['avg_rating'] * 20, "User Rating", 100, [60, 80]),  # Convert 1-5 to 0-100
            (drift_metrics['drift_score'], "Drift Score", 30, [8, 15]),
            (anomaly_metrics['anomaly_rate'], "Anomaly Rate", 20, [5, 10]),
            (metrics['avg_confidence'], "Confidence Score", 100, [75, 85])
        ]

        positions = [(1,1), (1,2), (1,3), (2,1), (2,2), (2,3)]

        for (value, title, max_val, thresholds), (row, col) in zip(gauge_configs, positions):
            fig.add_trace(
                go.Indicator(
                    mode="gauge+number",
                    value=value,
                    title={'text': title},
                    gauge={
                        'axis': {'range': [0, max_val]},
                        'bar': {'color': "darkblue"},
                        'steps': [
                            {'range': [0, thresholds[0]], 'color': "lightgray"},
                            {'range': [thresholds[0], thresholds[1]], 'color': "yellow"},
                            {'range': [thresholds[1], max_val], 'color': "lightgreen"}
                        ],
                        'threshold': {
                            'line': {'color': "red", 'width': 4},
                            'thickness': 0.75,
                            'value': thresholds[1]
                        }
                    },
                    domain={'row': row-1, 'column': col-1}
                ), row=row, col=col
            )

        fig.update_layout(
            title_text="📊 Performance Metrics Dashboard",
            height=500,
            template="plotly_white",
            title_x=0.5
        )

        return fig

    def create_drift_analysis(self):
        """Create drift analysis visualization"""

        # Calculate daily metrics
        daily_metrics = self.session_df.groupby('day').agg({
            'avg_confidence': 'mean',
            'resolved': 'mean',
            'used_fallback': 'mean',
            'session_id': 'count'
        }).reset_index()

        daily_metrics['confidence_pct'] = daily_metrics['avg_confidence'] * 100
        daily_metrics['resolution_pct'] = daily_metrics['resolved'] * 100
        daily_metrics['fallback_pct'] = daily_metrics['used_fallback'] * 100

        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Confidence Trend Over Time',
                'Resolution Rate Trend',
                'Fallback Rate Trend',
                'Daily Session Volume'
            )
        )

        # Confidence trend
        fig.add_trace(
            go.Scatter(
                x=daily_metrics['day'],
                y=daily_metrics['confidence_pct'],
                mode='lines+markers',
                name='Avg Confidence',
                line=dict(color=self.colors['primary'], width=3)
            ), row=1, col=1
        )

        # Resolution trend
        fig.add_trace(
            go.Scatter(
                x=daily_metrics['day'],
                y=daily_metrics['resolution_pct'],
                mode='lines+markers',
                name='Resolution Rate',
                line=dict(color=self.colors['success'], width=3)
            ), row=1, col=2
        )

        # Fallback trend
        fig.add_trace(
            go.Scatter(
                x=daily_metrics['day'],
                y=daily_metrics['fallback_pct'],
                mode='lines+markers',
                name='Fallback Rate',
                line=dict(color=self.colors['danger'], width=3)
            ), row=2, col=1
        )

        # Session volume
        fig.add_trace(
            go.Bar(
                x=daily_metrics['day'],
                y=daily_metrics['session_id'],
                name='Sessions',
                marker_color=self.colors['info']
            ), row=2, col=2
        )

        fig.update_layout(
            title_text="🔄 Performance Trends & Drift Analysis",
            height=600,
            showlegend=True,
            template="plotly_white",
            title_x=0.5
        )

        # Update axis labels
        fig.update_xaxes(title_text="Day", row=2, col=1)
        fig.update_xaxes(title_text="Day", row=2, col=2)
        fig.update_yaxes(title_text="Confidence %", row=1, col=1)
        fig.update_yaxes(title_text="Resolution %", row=1, col=2)
        fig.update_yaxes(title_text="Fallback %", row=2, col=1)
        fig.update_yaxes(title_text="Sessions", row=2, col=2)

        return fig

    def create_anomaly_detection(self):
        """Create anomaly detection dashboard"""

        # Calculate anomaly scores using simple method
        features = ['session_turns', 'avg_confidence', 'post_rating', 'response_time_ms']
        X = self.session_df[features]
        X_normalized = (X - X.mean()) / X.std()
        anomaly_scores = X_normalized.abs().max(axis=1)

        # Classify anomalies
        anomaly_threshold = 2.0
        self.session_df['is_anomaly'] = anomaly_scores > anomaly_threshold
        self.session_df['anomaly_score'] = anomaly_scores

        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Anomaly Score Distribution',
                'Daily Anomaly Rate',
                'Anomaly Characteristics',
                'Channel-wise Anomalies'
            )
        )

        # Anomaly score distribution
        normal_scores = anomaly_scores[~self.session_df['is_anomaly']]
        anomaly_scores_filtered = anomaly_scores[self.session_df['is_anomaly']]

        fig.add_trace(
            go.Histogram(
                x=normal_scores,
                name='Normal',
                marker_color=self.colors['success'],
                opacity=0.7,
                nbinsx=20
            ), row=1, col=1
        )

        fig.add_trace(
            go.Histogram(
                x=anomaly_scores_filtered,
                name='Anomalous',
                marker_color=self.colors['danger'],
                opacity=0.7,
                nbinsx=10
            ), row=1, col=1
        )

        # Daily anomaly rate
        daily_anomalies = self.session_df.groupby('day')['is_anomaly'].mean() * 100
        fig.add_trace(
            go.Bar(
                x=daily_anomalies.index,
                y=daily_anomalies.values,
                marker_color=self.colors['warning'],
                name='Anomaly Rate'
            ), row=1, col=2
        )

        # Anomaly characteristics
        normal_data = self.session_df[~self.session_df['is_anomaly']]
        anomaly_data = self.session_df[self.session_df['is_anomaly']]

        characteristics = ['session_turns', 'avg_confidence']
        char_names = ['Session Turns', 'Confidence']

        for i, (char, name) in enumerate(zip(characteristics, char_names)):
            fig.add_trace(
                go.Box(
                    y=normal_data[char],
                    name=f'Normal {name}',
                    marker_color=self.colors['success'],
                    showlegend=False
                ), row=2, col=1
            )

            fig.add_trace(
                go.Box(
                    y=anomaly_data[char],
                    name=f'Anomaly {name}',
                    marker_color=self.colors['danger'],
                    showlegend=False
                ), row=2, col=1
            )

        # Channel-wise anomalies
        channel_anomalies = self.session_df.groupby('channel')['is_anomaly'].mean() * 100
        fig.add_trace(
            go.Bar(
                x=channel_anomalies.index,
                y=channel_anomalies.values,
                marker_color=[self.colors['primary'], self.colors['success'],
                             self.colors['warning'], self.colors['danger']],
                name='Anomaly Rate by Channel'
            ), row=2, col=2
        )

        fig.update_layout(
            title_text="🚨 Anomaly Detection & Analysis",
            height=600,
            showlegend=True,
            template="plotly_white",
            title_x=0.5,
            barmode='overlay'
        )

        return fig

    def create_cross_platform_analysis(self):
        """Create cross-platform performance analysis"""

        channel_metrics = self.session_df.groupby('channel').agg({
            'session_id': 'count',
            'resolved': 'mean',
            'conversion': 'mean',
            'used_fallback': 'mean',
            'post_rating': 'mean',
            'avg_confidence': 'mean'
        }).rename(columns={'session_id': 'total_sessions'})

        channel_metrics['resolution_pct'] = channel_metrics['resolved'] * 100
        channel_metrics['conversion_pct'] = channel_metrics['conversion'] * 100
        channel_metrics['fallback_pct'] = channel_metrics['used_fallback'] * 100
        channel_metrics['confidence_pct'] = channel_metrics['avg_confidence'] * 100

        fig = make_subplots(
            rows=2, cols=3,
            subplot_titles=(
                'Resolution Rate by Channel',
                'Conversion Rate by Channel',
                'Fallback Rate by Channel',
                'User Rating by Channel',
                'Confidence by Channel',
                'Session Distribution'
            )
        )

        metrics = [
            ('resolution_pct', self.colors['success']),
            ('conversion_pct', self.colors['primary']),
            ('fallback_pct', self.colors['danger']),
            ('post_rating', self.colors['warning']),
            ('confidence_pct', self.colors['info']),
            ('total_sessions', self.colors['secondary'])
        ]

        for i, (metric, color) in enumerate(metrics):
            row = (i // 3) + 1
            col = (i % 3) + 1

            suffix = '%' if 'pct' in metric else ''

            fig.add_trace(
                go.Bar(
                    x=channel_metrics.index,
                    y=channel_metrics[metric],
                    marker_color=color,
                    text=channel_metrics[metric].round(1).astype(str) + suffix,
                    textposition='auto',
                    showlegend=False
                ), row=row, col=col
            )

        fig.update_layout(
            title_text="📱 Cross-Platform Performance Analysis",
            height=600,
            template="plotly_white",
            title_x=0.5
        )

        return fig

    def create_business_impact(self):
        """Create business impact analysis"""

        metrics = self.calculate_basic_metrics()

        # Business calculations
        conversion_sessions = self.session_df['conversion'].sum()
        fallback_sessions = self.session_df['used_fallback'].sum()

        estimated_revenue = conversion_sessions * AVG_CONVERSION_VALUE
        cost_savings = fallback_sessions * (AVG_HUMAN_CALL_DURATION_MIN / 60) * HUMAN_AGENT_HOURLY_COST

        monthly_hosting = HOSTING_COST_PER_YEAR / 12
        amortized_development = DEVELOPMENT_COST / 36
        total_monthly_cost = monthly_hosting + amortized_development

        monthly_roi = ((estimated_revenue + cost_savings) - total_monthly_cost) / total_monthly_cost * 100

        business_data = [
            ['Total Sessions', f"{metrics['total_sessions']:,}"],
            ['Conversion Sessions', f"{conversion_sessions:,}"],
            ['Estimated Monthly Revenue', f"€{estimated_revenue:,.0f}"],
            ['Monthly Cost Savings', f"€{cost_savings:,.0f}"],
            ['Monthly Total Cost', f"€{total_monthly_cost:,.0f}"],
            ['Monthly ROI', f"{monthly_roi:.1f}%"],
            ['Value per Session', f"€{(estimated_revenue/metrics['total_sessions']):.2f}"]
        ]

        fig = go.Figure(data=[go.Table(
            header=dict(
                values=['Business Metric', 'Value'],
                fill_color='lightsteelblue',
                align='center',
                font=dict(size=14, color='white')
            ),
            cells=dict(
                values=[[row[0] for row in business_data],
                        [row[1] for row in business_data]],
                fill_color=[['white', 'lightyellow'] * 4],
                align='center',
                font=dict(size=12)
            )
        )])

        fig.update_layout(
            title="💰 Business Impact Analysis",
            height=400,
            title_x=0.5
        )

        return fig

    def create_advanced_insights(self):
        """Create advanced insights and recommendations"""

        metrics = self.calculate_basic_metrics()
        drift_metrics = self.calculate_drift_metrics()
        anomaly_metrics = self.calculate_anomaly_metrics()

        insights = []

        # Resolution insights
        if metrics['resolution_rate'] > 75:
            insights.append(("✅ Resolution Rate", "Excellent performance - maintain current standards"))
        elif metrics['resolution_rate'] > 65:
            insights.append(("⚠️ Resolution Rate", "Good but needs monitoring - focus on fallback reduction"))
        else:
            insights.append(("🚨 Resolution Rate", "Needs improvement - review intent handling"))

        # Drift insights
        if drift_metrics['status'] == 'HIGH':
            insights.append(("🚨 Intent Drift", f"High drift detected ({drift_metrics['drift_score']:.1f}%) - consider model retraining"))
        elif drift_metrics['status'] == 'MODERATE':
            insights.append(("⚠️ Intent Drift", f"Moderate drift ({drift_metrics['drift_score']:.1f}%) - monitor closely"))
        else:
            insights.append(("✅ Intent Drift", f"Low drift ({drift_metrics['drift_score']:.1f}%) - stable performance"))

        # Anomaly insights
        if anomaly_metrics['status'] == 'HIGH':
            insights.append(("🚨 Anomaly Rate", f"High anomaly rate ({anomaly_metrics['anomaly_rate']:.1f}%) - investigate root causes"))
        elif anomaly_metrics['status'] == 'MODERATE':
            insights.append(("⚠️ Anomaly Rate", f"Moderate anomalies ({anomaly_metrics['anomaly_rate']:.1f}%) - review thresholds"))
        else:
            insights.append(("✅ Anomaly Rate", f"Normal operation ({anomaly_metrics['anomaly_rate']:.1f}%) - system stable"))

        # Channel insights
        channel_performance = self.session_df.groupby('channel')['resolved'].mean().sort_values(ascending=False)
        best_channel = channel_performance.index[0]
        worst_channel = channel_performance.index[-1]
        insights.append(("📱 Channel Performance", f"Best: {best_channel}, Needs attention: {worst_channel}"))

        fig = go.Figure(data=[go.Table(
            header=dict(
                values=['Metric', 'Insight & Recommendation'],
                fill_color='lightsteelblue',
                align='left',
                font=dict(size=14, color='white')
            ),
            cells=dict(
                values=[[insight[0] for insight in insights],
                        [insight[1] for insight in insights]],
                fill_color=[['white', 'lightyellow'] * 3],
                align='left',
                font=dict(size=12)
            )
        )])

        fig.update_layout(
            title="🎯 Advanced Insights & Recommendations",
            height=350,
            title_x=0.5
        )

        return fig

    def create_comprehensive_dashboard(self):
        """Create and display the complete dashboard"""

        print("🏦 ADVANCED BANKING CHATBOT ANALYTICS DASHBOARD")
        print("=" * 70)
        print("Loading interactive analytics with gauges, drift detection & anomaly monitoring...")
        print("=" * 70)

        # Display all components
        self.create_executive_dashboard().show()
        self.create_performance_gauges().show()
        self.create_drift_analysis().show()
        self.create_anomaly_detection().show()
        self.create_cross_platform_analysis().show()
        self.create_business_impact().show()
        self.create_advanced_insights().show()

        print("\n" + "="*80)
        print("✅ DASHBOARD COMPONENTS LOADED SUCCESSFULLY:")
        print("• Executive Dashboard: Key metrics and channel distribution")
        print("• Performance Gauges: 6 interactive gauges for real-time monitoring")
        print("• Drift Analysis: Confidence and performance trends over time")
        print("• Anomaly Detection: Outlier identification and analysis")
        print("• Cross-Platform: Channel performance comparison")
        print("• Business Impact: ROI calculations and value demonstration")
        print("• Advanced Insights: AI-powered recommendations")
        print("="*80)

# =============================================
# RUN THE FIXED DASHBOARD
# =============================================

print("\n🚀 INITIALIZING FIXED ADVANCED DASHBOARD...")
print("=" * 60)

# Create and run the dashboard
dashboard = FixedAdvancedDashboard(session_df, message_df)
dashboard.create_comprehensive_dashboard()

print("\n🎉 DASHBOARD EXECUTION COMPLETED SUCCESSFULLY!")
print("All charts and analytics are now displayed above.")

🔄 Generating banking chatbot data...
✅ Generated 2000 sessions and 10061 messages
Session data sample:


,session_id,user_id,channel,day,timestamp,resolved,used_fallback,conversion,post_rating,session_turns,true_intent,avg_confidence,response_time_ms,anomaly_score
0,session_0,user_84810,web,0,2025-09-19T20:56:17.629266,True,False,False,4,2,greet,0.892323,571.261834,5.503557
1,session_1,user_98196,whatsapp,1,2025-09-20T20:56:17.629266,True,False,False,3,4,transfer_funds,0.819642,521.726456,0.361015



🚀 INITIALIZING FIXED ADVANCED DASHBOARD...
🏦 ADVANCED BANKING CHATBOT ANALYTICS DASHBOARD
Loading interactive analytics with gauges, drift detection & anomaly monitoring...



✅ DASHBOARD COMPONENTS LOADED SUCCESSFULLY:
• Executive Dashboard: Key metrics and channel distribution
• Performance Gauges: 6 interactive gauges for real-time monitoring
• Drift Analysis: Confidence and performance trends over time
• Anomaly Detection: Outlier identification and analysis
• Cross-Platform: Channel performance comparison
• Business Impact: ROI calculations and value demonstration
• Advanced Insights: AI-powered recommendations

🎉 DASHBOARD EXECUTION COMPLETED SUCCESSFULLY!
All charts and analytics are now displayed above.


In [ ]:
# =============================================
# COMPLETE GRADIO DASHBOARD FOR BANKING CHATBOT ANALYTICS MADE WITH GRADIO UI
# =============================================

import gradio as gr
import random
import uuid
import json
from datetime import datetime, timedelta
from collections import Counter, defaultdict
import math

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

random.seed(42)
np.random.seed(42)

# ----------------------------
# Configuration
# ----------------------------
NUM_SESSIONS = 1500  # Reduced for Gradio performance
BASE_TIME = datetime.utcnow() - timedelta(days=30)
HUMAN_AGENT_HOURLY_COST = 25.0
AVG_HUMAN_CALL_DURATION_MIN = 7
DEVELOPMENT_COST = 5000.0
HOSTING_COST_PER_YEAR = 800.0
AVG_CONVERSION_VALUE = 50.0

INTENTS = [
    "greet", "check_balance", "transactions", "transfer_funds",
    "open_account", "loan_inquiry", "card_block", "faq_interest_rates",
    "update_details", "goodbye", "small_talk"
]

CONVERSION_INTENTS = {"transfer_funds", "open_account"}
CHANNELS = ["web", "mobile_app", "whatsapp", "facebook_messenger"]

# ----------------------------
# Data Generation
# ----------------------------

def generate_chatbot_data(num_sessions=1500):
    """Generate banking chatbot data"""
    print("🔄 Generating chatbot data...")

    sessions = []
    messages = []

    for i in range(num_sessions):
        session_id = f"session_{i}"
        user_id = f"user_{random.randint(1000, 99999)}"
        channel = random.choice(CHANNELS)
        day = i % 30

        # Create temporal patterns
        if day < 10:
            resolution_prob = 0.78
            confidence_mean = 0.87
            fallback_prob = 0.10
        elif day < 20:
            resolution_prob = 0.72
            confidence_mean = 0.82
            fallback_prob = 0.15
        else:
            resolution_prob = 0.65
            confidence_mean = 0.75
            fallback_prob = 0.22

        # Session data
        session_data = {
            'session_id': session_id,
            'user_id': user_id,
            'channel': channel,
            'day': day,
            'timestamp': (BASE_TIME + timedelta(days=day)).isoformat(),
            'resolved': np.random.random() < resolution_prob,
            'used_fallback': np.random.random() < fallback_prob,
            'conversion': np.random.random() < 0.23,
            'post_rating': np.random.choice([1, 2, 3, 4, 5], p=[0.05, 0.1, 0.2, 0.4, 0.25]),
            'session_turns': max(2, int(np.random.poisson(5))),
            'true_intent': random.choice(INTENTS),
            'avg_confidence': np.clip(np.random.normal(confidence_mean, 0.08), 0.1, 0.99),
            'response_time_ms': np.random.normal(450, 120)
        }

        # Anomaly score
        session_data['anomaly_score'] = np.random.exponential(0.5)
        if np.random.random() < 0.08:
            session_data['anomaly_score'] = 2 + np.random.exponential(1)

        sessions.append(session_data)

        # Messages
        num_messages = session_data['session_turns']
        for turn in range(num_messages):
            messages.append({
                'session_id': session_id,
                'message_id': f"msg_{i}_{turn}",
                'turn': turn,
                'true_intent': session_data['true_intent'],
                'predicted_intent': session_data['true_intent'],
                'confidence': np.clip(np.random.normal(confidence_mean, 0.1), 0.1, 0.99),
                'response_time_ms': np.random.normal(450, 120)
            })

    return pd.DataFrame(sessions), pd.DataFrame(messages)

# Generate data once at startup
print("📊 Loading chatbot data...")
session_df, message_df = generate_chatbot_data(1500)
print(f"✅ Generated {len(session_df)} sessions")

# ----------------------------
# Dashboard Class
# ----------------------------

class GradioBankingDashboard:
    """Banking Chatbot Dashboard for Gradio"""

    def __init__(self, session_df, message_df):
        self.session_df = session_df
        self.message_df = message_df
        self.colors = {
            'primary': '#1f77b4', 'success': '#2ca02c', 'warning': '#ff7f0e',
            'danger': '#d62728', 'info': '#17becf', 'secondary': '#7f7f7f'
        }

    def calculate_metrics(self):
        """Calculate all metrics"""
        total_sessions = len(self.session_df)
        resolution_rate = (self.session_df['resolved'].sum() / total_sessions) * 100
        conversion_rate = (self.session_df['conversion'].sum() / total_sessions) * 100
        fallback_rate = (self.session_df['used_fallback'].sum() / total_sessions) * 100
        avg_rating = self.session_df['post_rating'].mean()
        avg_confidence = self.session_df['avg_confidence'].mean() * 100

        # Simple drift calculation
        daily_confidence = self.session_df.groupby('day')['avg_confidence'].mean()
        baseline = daily_confidence.iloc[:7].mean()
        current = daily_confidence.iloc[-1]
        drift_score = abs(current - baseline) * 100

        # Simple anomaly detection
        features = ['session_turns', 'avg_confidence', 'post_rating', 'response_time_ms']
        X = self.session_df[features]
        X_normalized = (X - X.mean()) / X.std()
        anomaly_scores = X_normalized.abs().max(axis=1)
        anomaly_rate = (anomaly_scores > 2.0).sum() / len(self.session_df) * 100

        return {
            'total_sessions': total_sessions,
            'resolution_rate': resolution_rate,
            'conversion_rate': conversion_rate,
            'fallback_rate': fallback_rate,
            'avg_rating': avg_rating,
            'avg_confidence': avg_confidence,
            'drift_score': drift_score,
            'anomaly_rate': anomaly_rate
        }

    def create_executive_summary(self):
        """Create executive summary"""
        metrics = self.calculate_metrics()

        fig = make_subplots(
            rows=2, cols=3,
            specs=[
                [{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}],
                [{"type": "indicator"}, {"type": "indicator"}, {"type": "bar"}]
            ]
        )

        # Main KPIs
        main_kpis = [
            (metrics['total_sessions'], "Total Sessions", "", 1, 1),
            (metrics['resolution_rate'], "Resolution Rate", "%", 1, 2),
            (metrics['conversion_rate'], "Conversion Rate", "%", 1, 3),
            (metrics['avg_rating'], "Avg Rating", "/5", 2, 1),
            (metrics['drift_score'], "Drift Score", "%", 2, 2)
        ]

        for value, title, suffix, row, col in main_kpis:
            fig.add_trace(
                go.Indicator(
                    mode="number",
                    value=value,
                    number={'suffix': suffix, 'font': {'size': 24}},
                    title={'text': title, 'font': {'size': 14}},
                    domain={'row': row-1, 'column': col-1}
                ), row=row, col=col
            )

        # Channel distribution
        channel_counts = self.session_df['channel'].value_counts()
        fig.add_trace(
            go.Bar(
                x=channel_counts.index,
                y=channel_counts.values,
                marker_color=[self.colors['primary'], self.colors['success'],
                             self.colors['warning'], self.colors['danger']],
                text=channel_counts.values,
                textposition='auto'
            ), row=2, col=3
        )

        fig.update_layout(
            title_text="🏦 Banking Chatbot - Executive Dashboard",
            height=500,
            showlegend=False,
            template="plotly_white"
        )

        return fig

    def create_performance_gauges(self):
        """Create performance gauges"""
        metrics = self.calculate_metrics()

        fig = make_subplots(
            rows=2, cols=3,
            specs=[[{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}],
                   [{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}]],
            subplot_titles=('Resolution Rate', 'Conversion Rate', 'User Rating',
                           'Drift Score', 'Anomaly Rate', 'Confidence Score')
        )

        gauge_configs = [
            (metrics['resolution_rate'], "Resolution Rate", 100, [70, 85]),
            (metrics['conversion_rate'], "Conversion Rate", 50, [15, 30]),
            (metrics['avg_rating'] * 20, "User Rating", 100, [60, 80]),
            (metrics['drift_score'], "Drift Score", 30, [8, 15]),
            (metrics['anomaly_rate'], "Anomaly Rate", 20, [5, 10]),
            (metrics['avg_confidence'], "Confidence Score", 100, [75, 85])
        ]

        positions = [(1,1), (1,2), (1,3), (2,1), (2,2), (2,3)]

        for (value, title, max_val, thresholds), (row, col) in zip(gauge_configs, positions):
            fig.add_trace(
                go.Indicator(
                    mode="gauge+number",
                    value=value,
                    title={'text': title},
                    gauge={
                        'axis': {'range': [0, max_val]},
                        'bar': {'color': "darkblue"},
                        'steps': [
                            {'range': [0, thresholds[0]], 'color': "lightgray"},
                            {'range': [thresholds[0], thresholds[1]], 'color': "yellow"},
                            {'range': [thresholds[1], max_val], 'color': "lightgreen"}
                        ],
                        'threshold': {
                            'line': {'color': "red", 'width': 4},
                            'thickness': 0.75,
                            'value': thresholds[1]
                        }
                    },
                    domain={'row': row-1, 'column': col-1}
                ), row=row, col=col
            )

        fig.update_layout(
            title_text="📊 Performance Metrics Dashboard",
            height=500,
            template="plotly_white"
        )

        return fig

    def create_drift_analysis(self):
        """Create drift analysis"""
        daily_metrics = self.session_df.groupby('day').agg({
            'avg_confidence': 'mean',
            'resolved': 'mean',
            'used_fallback': 'mean',
            'session_id': 'count'
        }).reset_index()

        daily_metrics['confidence_pct'] = daily_metrics['avg_confidence'] * 100
        daily_metrics['resolution_pct'] = daily_metrics['resolved'] * 100
        daily_metrics['fallback_pct'] = daily_metrics['used_fallback'] * 100

        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Confidence Trend Over Time',
                'Resolution Rate Trend',
                'Fallback Rate Trend',
                'Daily Session Volume'
            )
        )

        # Confidence trend
        fig.add_trace(
            go.Scatter(
                x=daily_metrics['day'],
                y=daily_metrics['confidence_pct'],
                mode='lines+markers',
                name='Avg Confidence',
                line=dict(color=self.colors['primary'], width=3)
            ), row=1, col=1
        )

        # Resolution trend
        fig.add_trace(
            go.Scatter(
                x=daily_metrics['day'],
                y=daily_metrics['resolution_pct'],
                mode='lines+markers',
                name='Resolution Rate',
                line=dict(color=self.colors['success'], width=3)
            ), row=1, col=2
        )

        # Fallback trend
        fig.add_trace(
            go.Scatter(
                x=daily_metrics['day'],
                y=daily_metrics['fallback_pct'],
                mode='lines+markers',
                name='Fallback Rate',
                line=dict(color=self.colors['danger'], width=3)
            ), row=2, col=1
        )

        # Session volume
        fig.add_trace(
            go.Bar(
                x=daily_metrics['day'],
                y=daily_metrics['session_id'],
                name='Sessions',
                marker_color=self.colors['info']
            ), row=2, col=2
        )

        fig.update_layout(
            title_text="🔄 Performance Trends & Drift Analysis",
            height=600,
            showlegend=True,
            template="plotly_white"
        )

        fig.update_xaxes(title_text="Day", row=2, col=1)
        fig.update_xaxes(title_text="Day", row=2, col=2)
        fig.update_yaxes(title_text="Confidence %", row=1, col=1)
        fig.update_yaxes(title_text="Resolution %", row=1, col=2)
        fig.update_yaxes(title_text="Fallback %", row=2, col=1)
        fig.update_yaxes(title_text="Sessions", row=2, col=2)

        return fig

    def create_anomaly_detection(self):
        """Create anomaly detection"""
        features = ['session_turns', 'avg_confidence', 'post_rating', 'response_time_ms']
        X = self.session_df[features]
        X_normalized = (X - X.mean()) / X.std()
        anomaly_scores = X_normalized.abs().max(axis=1)

        self.session_df['is_anomaly'] = anomaly_scores > 2.0
        self.session_df['anomaly_score'] = anomaly_scores

        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Anomaly Score Distribution',
                'Daily Anomaly Rate',
                'Anomaly Characteristics',
                'Channel-wise Anomalies'
            )
        )

        # Anomaly score distribution
        normal_scores = anomaly_scores[~self.session_df['is_anomaly']]
        anomaly_scores_filtered = anomaly_scores[self.session_df['is_anomaly']]

        fig.add_trace(
            go.Histogram(
                x=normal_scores,
                name='Normal',
                marker_color=self.colors['success'],
                opacity=0.7,
                nbinsx=20
            ), row=1, col=1
        )

        fig.add_trace(
            go.Histogram(
                x=anomaly_scores_filtered,
                name='Anomalous',
                marker_color=self.colors['danger'],
                opacity=0.7,
                nbinsx=10
            ), row=1, col=1
        )

        # Daily anomaly rate
        daily_anomalies = self.session_df.groupby('day')['is_anomaly'].mean() * 100
        fig.add_trace(
            go.Bar(
                x=daily_anomalies.index,
                y=daily_anomalies.values,
                marker_color=self.colors['warning'],
                name='Anomaly Rate'
            ), row=1, col=2
        )

        # Anomaly characteristics
        normal_data = self.session_df[~self.session_df['is_anomaly']]
        anomaly_data = self.session_df[self.session_df['is_anomaly']]

        characteristics = ['session_turns', 'avg_confidence']

        for i, char in enumerate(characteristics):
            fig.add_trace(
                go.Box(
                    y=normal_data[char],
                    name=f'Normal {char}',
                    marker_color=self.colors['success'],
                    showlegend=False
                ), row=2, col=1
            )

            fig.add_trace(
                go.Box(
                    y=anomaly_data[char],
                    name=f'Anomaly {char}',
                    marker_color=self.colors['danger'],
                    showlegend=False
                ), row=2, col=1
            )

        # Channel-wise anomalies
        channel_anomalies = self.session_df.groupby('channel')['is_anomaly'].mean() * 100
        fig.add_trace(
            go.Bar(
                x=channel_anomalies.index,
                y=channel_anomalies.values,
                marker_color=[self.colors['primary'], self.colors['success'],
                             self.colors['warning'], self.colors['danger']],
                name='Anomaly Rate by Channel'
            ), row=2, col=2
        )

        fig.update_layout(
            title_text="🚨 Anomaly Detection & Analysis",
            height=600,
            showlegend=True,
            template="plotly_white",
            barmode='overlay'
        )

        return fig

    def create_cross_platform_analysis(self):
        """Create cross-platform analysis"""
        channel_metrics = self.session_df.groupby('channel').agg({
            'session_id': 'count',
            'resolved': 'mean',
            'conversion': 'mean',
            'used_fallback': 'mean',
            'post_rating': 'mean',
            'avg_confidence': 'mean'
        }).rename(columns={'session_id': 'total_sessions'})

        channel_metrics['resolution_pct'] = channel_metrics['resolved'] * 100
        channel_metrics['conversion_pct'] = channel_metrics['conversion'] * 100
        channel_metrics['fallback_pct'] = channel_metrics['used_fallback'] * 100
        channel_metrics['confidence_pct'] = channel_metrics['avg_confidence'] * 100

        fig = make_subplots(
            rows=2, cols=3,
            subplot_titles=(
                'Resolution Rate by Channel',
                'Conversion Rate by Channel',
                'Fallback Rate by Channel',
                'User Rating by Channel',
                'Confidence by Channel',
                'Session Distribution'
            )
        )

        metrics = [
            ('resolution_pct', self.colors['success']),
            ('conversion_pct', self.colors['primary']),
            ('fallback_pct', self.colors['danger']),
            ('post_rating', self.colors['warning']),
            ('confidence_pct', self.colors['info']),
            ('total_sessions', self.colors['secondary'])
        ]

        for i, (metric, color) in enumerate(metrics):
            row = (i // 3) + 1
            col = (i % 3) + 1

            suffix = '%' if 'pct' in metric else ''

            fig.add_trace(
                go.Bar(
                    x=channel_metrics.index,
                    y=channel_metrics[metric],
                    marker_color=color,
                    text=channel_metrics[metric].round(1).astype(str) + suffix,
                    textposition='auto',
                    showlegend=False
                ), row=row, col=col
            )

        fig.update_layout(
            title_text="📱 Cross-Platform Performance Analysis",
            height=600,
            template="plotly_white"
        )

        return fig

    def create_business_impact(self):
        """Create business impact analysis"""
        metrics = self.calculate_metrics()

        conversion_sessions = self.session_df['conversion'].sum()
        fallback_sessions = self.session_df['used_fallback'].sum()

        estimated_revenue = conversion_sessions * AVG_CONVERSION_VALUE
        cost_savings = fallback_sessions * (AVG_HUMAN_CALL_DURATION_MIN / 60) * HUMAN_AGENT_HOURLY_COST

        monthly_hosting = HOSTING_COST_PER_YEAR / 12
        amortized_development = DEVELOPMENT_COST / 36
        total_monthly_cost = monthly_hosting + amortized_development

        monthly_roi = ((estimated_revenue + cost_savings) - total_monthly_cost) / total_monthly_cost * 100

        business_data = [
            ['Total Sessions', f"{metrics['total_sessions']:,}"],
            ['Conversion Sessions', f"{conversion_sessions:,}"],
            ['Estimated Monthly Revenue', f"€{estimated_revenue:,.0f}"],
            ['Monthly Cost Savings', f"€{cost_savings:,.0f}"],
            ['Monthly Total Cost', f"€{total_monthly_cost:,.0f}"],
            ['Monthly ROI', f"{monthly_roi:.1f}%"],
            ['Value per Session', f"€{(estimated_revenue/metrics['total_sessions']):.2f}"]
        ]

        fig = go.Figure(data=[go.Table(
            header=dict(
                values=['Business Metric', 'Value'],
                fill_color='lightsteelblue',
                align='center',
                font=dict(size=14, color='white')
            ),
            cells=dict(
                values=[[row[0] for row in business_data],
                        [row[1] for row in business_data]],
                fill_color=[['white', 'lightyellow'] * 4],
                align='center',
                font=dict(size=12)
            )
        )])

        fig.update_layout(
            title="💰 Business Impact Analysis",
            height=400
        )

        return fig

# ----------------------------
# Gradio UI Setup
# ----------------------------

# Create dashboard instance
dashboard = GradioBankingDashboard(session_df, message_df)

def create_all_plots():
    """Create all plots for the dashboard"""
    try:
        print("🔄 Generating plots...")

        exec_summary = dashboard.create_executive_summary()
        performance_gauges = dashboard.create_performance_gauges()
        drift_analysis = dashboard.create_drift_analysis()
        anomaly_detection = dashboard.create_anomaly_detection()
        cross_platform = dashboard.create_cross_platform_analysis()
        business_impact = dashboard.create_business_impact()

        print("✅ All plots generated successfully!")

        return (exec_summary, performance_gauges, drift_analysis,
                anomaly_detection, cross_platform, business_impact)

    except Exception as e:
        print(f"❌ Error generating plots: {e}")
        # Return empty figures in case of error
        empty_fig = go.Figure()
        empty_fig.update_layout(title="Error loading plot")
        return (empty_fig, empty_fig, empty_fig, empty_fig, empty_fig, empty_fig)

# Create Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), title="Banking Chatbot Analytics") as demo:
    gr.Markdown("""
    # 🏦 Banking Chatbot Analytics Dashboard
    *Real-time monitoring of chatbot performance, intent drift, and business impact*
    """)

    with gr.Row():
        gr.Markdown("""
        ### 📊 Dashboard Overview
        This interactive dashboard provides comprehensive analytics for your banking chatbot, including performance metrics, drift detection, and anomaly monitoring.
        """)

    with gr.Tab("📈 Executive Summary"):
        exec_plot = gr.Plot(label="Executive Summary")

    with gr.Tab("🎯 Performance Gauges"):
        gauge_plot = gr.Plot(label="Performance Gauges")

    with gr.Tab("🔄 Drift Analysis"):
        drift_plot = gr.Plot(label="Drift Analysis")

    with gr.Tab("🚨 Anomaly Detection"):
        anomaly_plot = gr.Plot(label="Anomaly Detection")

    with gr.Tab("📱 Cross-Platform"):
        cross_platform_plot = gr.Plot(label="Cross-Platform Analysis")

    with gr.Tab("💰 Business Impact"):
        business_plot = gr.Plot(label="Business Impact")

    # Load all plots when the demo starts
    demo.load(
        fn=create_all_plots,
        inputs=None,
        outputs=[exec_plot, gauge_plot, drift_plot, anomaly_plot, cross_platform_plot, business_plot]
    )

    # Refresh button
    with gr.Row():
        refresh_btn = gr.Button("🔄 Refresh Dashboard", variant="primary")
        refresh_btn.click(
            fn=create_all_plots,
            inputs=None,
            outputs=[exec_plot, gauge_plot, drift_plot, anomaly_plot, cross_platform_plot, business_plot]
        )

    gr.Markdown("""
    ---
    **Dashboard Features:**
    - **Real-time Performance Monitoring** with interactive gauges
    - **Intent Drift Detection** to track model degradation
    - **Anomaly Detection** for identifying unusual patterns
    - **Cross-Platform Analysis** comparing channel performance
    - **Business Impact** calculations showing ROI and value
    """)

# ----------------------------
# Launch the Dashboard
# ----------------------------

if __name__ == "__main__":
    print("🚀 Launching Gradio Dashboard...")
    print("📊 Dashboard will open in your browser shortly...")
    print("🔗 If running in Colab, check the public URL provided by Gradio")

    # Launch with sharing enabled for public access
    demo.launch(
        share=True,  # Creates public URL
        inbrowser=True,  # Opens in browser
        show_error=True,  # Shows detailed errors
        debug=True  # Enable debugging
    )

📊 Loading chatbot data...
🔄 Generating chatbot data...
✅ Generated 1500 sessions
🚀 Launching Gradio Dashboard...
📊 Dashboard will open in your browser shortly...
🔗 If running in Colab, check the public URL provided by Gradio
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b078e72e6c685f6c71.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🔄 Generating plots...
✅ All plots generated successfully!
🔄 Generating plots...
✅ All plots generated successfully!
🔄 Generating plots...
✅ All plots generated successfully!
🔄 Generating plots...
✅ All plots generated successfully!
🔄 Generating plots...
✅ All plots generated successfully!
🔄 Generating plots...
✅ All plots generated successfully!
